In [ ]:
!pip install automl-alex

In [ ]:
# import lightgbm as lgb
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import seaborn as sns
import warnings
import os
import time
import copy
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import shutil
import glob
from IPython.display import clear_output 

In [ ]:
import numpy as np

import math
import os
import matplotlib.pyplot as plt
import datetime
from datetime import date
from sklearn.metrics import mean_absolute_error

# from lightgbm import LGBMRegressor
# from sklearn.model_selection import KFold
!pip install workalendar -q
from workalendar.asia import SouthKorea

     |████████████████████████████████| 194kB 7.0MB/s 
     |████████████████████████████████| 16.9MB 397kB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 378kB 43.8MB/s 
     |████████████████████████████████| 40kB 5.0MB/s 
     |████████████████████████████████| 266kB 48.4MB/s 


In [ ]:
# https://dacon.io/competitions/official/235697/codeshare/2363?page=1&dtype=recent&ptype=pub
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)  # type: ignore
    # torch.backends.cudnn.deterministic = True  # type: ignore
    # torch.backends.cudnn.benchmark = True  # type: ignore

seed_everything()

In [ ]:
# /content/drive/MyDrive/daycon_data/전력사용량예측/energy.zip

os.makedirs('./data')
os.makedirs('./data/all')
# os.makedirs('./data/train')
# os.makedirs('./data/val')
!unzip "/content/drive/MyDrive/daycon_data/전력사용량예측/energy.zip" -d "./data/all/" 

clear_output()

In [ ]:
train=pd.read_csv('/content/data/all/energy/train.csv', encoding='cp949')
test=pd.read_csv('/content/data/all/energy/test.csv', encoding='cp949')
submission=pd.read_csv('/content/data/all/energy/sample_submission.csv', encoding='cp949')

In [ ]:
train.columns = ['num', 'date_time', '전력사용량', '기온', '풍속', '습도', '강수량', '일조', '비전기냉방설비운영', '태양광보유']
test.columns = ['num', 'date_time', '기온', '풍속', '습도', '강수량', '일조', '비전기냉방설비운영', '태양광보유']

train

,num,date_time,전력사용량,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0


In [ ]:
test

,num,date_time,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,NaN,NaN
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,NaN,NaN
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
holidays = pd.Series(np.array(SouthKorea().holidays(2020))[:, 0])
holidays

0     2020-01-01
1     2020-01-24
2     2020-01-25
3     2020-01-26
4     2020-03-01
5     2020-04-30
6     2020-05-05
7     2020-06-06
8     2020-08-15
9     2020-09-30
10    2020-10-01
11    2020-10-02
12    2020-10-03
13    2020-10-09
14    2020-12-25
dtype: object

In [ ]:
#건물별로 '비전기냉방설비운영'과 '태양광보유'를 판단해 test set의 결측치를 보간해줍니다
train[['num', '비전기냉방설비운영','태양광보유']]
ice={}
hot={}
count=0
for i in range(0, len(train), len(train)//60):
    count +=1
    ice[count]=train.loc[i,'비전기냉방설비운영']
    hot[count]=train.loc[i,'태양광보유']

for i in tqdm(range(len(test))):
    test.loc[i, '비전기냉방설비운영']=ice[test['num'][i]]
    test.loc[i, '태양광보유']=hot[test['num'][i]]

100%|██████████| 10080/10080 [00:06<00:00, 1514.91it/s]


In [ ]:
test

,num,date_time,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0.0,0.0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0.0,0.0
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0.0,0.0
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1.0,1.0
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1.0,1.0
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1.0,1.0
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1.0,1.0


# 무슨 요일인지 변수 추가 필요함

In [ ]:
!pip install pendulum

     |████████████████████████████████| 163kB 5.2MB/s 
     |████████████████████████████████| 491kB 8.3MB/s 


In [ ]:
import pendulum

In [ ]:
%%time

#시간 변수와 요일 변수를 추가해봅니다.
def time(x):
    return int(x[-2:])

# 요일
def weekday(x):
    return pd.to_datetime(x[:10]).weekday()

def month(x):
    return pd.to_datetime(x[:10]).month

def week(x):
    return pd.to_datetime(x[:10]).week

def day(x):
    return pd.to_datetime(x[:10]).day

# 공휴일 - 수정 완료 - 그런데 test일때는 존재하지 않는 공휴일을 반영할 필요가 있는지?
def holiday(x):
    return int(pd.to_datetime(x[:10]).date() in list(holidays))

def weekend(x):
    if x == 0 or x == 1 or x == 2 or x == 3 or x==4:
        return 0
    else:
        return 1

def time_interval(x):
  return 2207-(( pd.to_datetime('2020-08-31 23') - pd.to_datetime(x) ).days*24 + ( pd.to_datetime('2020-08-31 23') - pd.to_datetime(x) ).seconds//3600)

def week_of_month(x):
  dt = pendulum.parse(x)
  return dt.week_of_month

def tomorrow(x):
    return (pd.to_datetime(x[:10]).date() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

def yesterday(x):
    return (pd.to_datetime(x[:10]).date() + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")

train['time']=train['date_time'].apply(lambda x: time(x))
test['time']=test['date_time'].apply(lambda x: time(x))

# weekday : 요일
train['weekday']=train['date_time'].apply(lambda x :weekday(x))
test['weekday']=test['date_time'].apply(lambda x :weekday(x))


# 달이랑 주가 왜 빠졌지?
# train에 8월달이 없어서 그런가

train['month']=train['date_time'].apply(lambda x :month(x))
test['month']=test['date_time'].apply(lambda x :month(x))
train['week']=train['date_time'].apply(lambda x :week(x))
test['week']=test['date_time'].apply(lambda x :week(x))
train['day']=train['date_time'].apply(lambda x :day(x))
test['day']=test['date_time'].apply(lambda x :day(x))

# week 가 한 달의 몇 주 째인지 안나오고 합산으로 나옴.
train['week_of_month']=train['date_time'].apply(lambda x :week_of_month(x))
test['week_of_month']=test['date_time'].apply(lambda x :week_of_month(x))

# 공휴일 - 수정필요
# https://github.com/league3236/shholiday
train['holiday']=train['date_time'].apply(lambda x :holiday(x))
test['holiday']=test['date_time'].apply(lambda x :holiday(x))

# 주말
train['weekend']=train['weekday'].apply(lambda x :weekend(x))
test['weekend']=test['weekday'].apply(lambda x :weekend(x))
# train['date']=train['date_time'].apply(lambda x :date(x))
# test['date']=test['date_time'].apply(lambda x :date(x))

# 시간간격 time interval
train['시간간격']=train['date_time'].apply(lambda x :time_interval(x))
test['시간간격']=test['date_time'].apply(lambda x :time_interval(x))

# train['tomorrow']=train['date_time'].apply(lambda x :tomorrow(x))
# test['tomorrow']=test['date_time'].apply(lambda x :tomorrow(x))
# train['yesterday']=train['date_time'].apply(lambda x :yesterday(x))
# test['yesterday']=test['date_time'].apply(lambda x :yesterday(x))

CPU times: user 1min 54s, sys: 400 ms, total: 1min 55s
Wall time: 1min 54s


# test 결측치 선형 보간.

상식적으로 기온의 경우, 비선형일 것인데, 선형으로 해도 될까 의문.

이었으나 퍼블릭 점수 기준, 선형으로 하는게 제일 좋았던것으로 기억합니다.

In [ ]:
test =test.interpolate(method='values')

# #기온 결측치 채우기
# test['기온(°C)'] = test['기온(°C)'].interpolate(method='quadratic')
# #마지막 na 채우기
# test['기온(°C)'] = test['기온(°C)'].interpolate(method='linear')

# #풍속 결측치 채우기
# test['풍속(m/s)'] = test['풍속(m/s)'].interpolate(method='linear')

# #습도 결측치 채우기
# test['습도(%)'] = test['습도(%)'].interpolate(method='quadratic')
# #마지막 na 채우기
# test['습도(%)'] = test['습도(%)'].interpolate(method='linear')

# #강수량 결측치 채우기
# test['강수량(mm, 6시간)'] = test['강수량(mm, 6시간)'].interpolate(method='linear')

# ----------------------------# ----------------------------# ----------------------------# ----------------------------# ----------------------------# ----------------------------# ----------------------------# ----------------------------# ----------------------------

# #기온 결측치 채우기
# test['기온'] = test['기온'].interpolate(method='quadratic')
# #마지막 na 채우기
# test['기온'] = test['기온'].interpolate(method='linear')

# #풍속 결측치 채우기
# test['풍속'] = test['풍속'].interpolate(method='linear')

# #습도 결측치 채우기
# test['습도'] = test['습도'].interpolate(method='quadratic')
# #마지막 na 채우기
# test['습도'] = test['습도'].interpolate(method='linear')

# #강수량 결측치 채우기
# test['강수량'] = test['강수량'].interpolate(method='linear')

# test['일조'] = test['일조'].interpolate(method='linear')
test

,num,date_time,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격
0,1,2020-08-25 00,27.800000,1.500000,74.000000,0.0,0.000000,0.0,0.0,0,1,8,35,25,5,0,0,2040
1,1,2020-08-25 01,27.633333,1.366667,75.333333,0.0,0.000000,0.0,0.0,1,1,8,35,25,5,0,0,2041
2,1,2020-08-25 02,27.466667,1.233333,76.666667,0.0,0.000000,0.0,0.0,2,1,8,35,25,5,0,0,2042
3,1,2020-08-25 03,27.300000,1.100000,78.000000,0.0,0.000000,0.0,0.0,3,1,8,35,25,5,0,0,2043
4,1,2020-08-25 04,26.900000,1.166667,79.666667,0.0,0.000000,0.0,0.0,4,1,8,35,25,5,0,0,2044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,28.633333,3.566667,66.000000,0.0,0.533333,1.0,1.0,19,0,8,36,31,6,0,0,2203
10076,60,2020-08-31 20,28.266667,3.833333,67.000000,0.0,0.266667,1.0,1.0,20,0,8,36,31,6,0,0,2204
10077,60,2020-08-31 21,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,21,0,8,36,31,6,0,0,2205
10078,60,2020-08-31 22,27.900000,4.100000,68.000000,0.0,0.000000,1.0,1.0,22,0,8,36,31,6,0,0,2206


In [ ]:
train.drop('date_time', axis=1, inplace=True)
test.drop('date_time', axis=1, inplace=True)

In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,6,23,1,1,0,0,0
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,6,23,1,1,0,0,1
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,6,23,1,1,0,0,2
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,0,6,23,1,1,0,0,3
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,0,6,23,1,1,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,19,0,8,35,24,5,0,0,2035
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,20,0,8,35,24,5,0,0,2036
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,21,0,8,35,24,5,0,0,2037
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,22,0,8,35,24,5,0,0,2038


## 비전기냉방설비운영&태양광보유 combi & one hot

combi 속성은 냉방설비와 태양광 보유한 경우의 수를 독립적으로... 고려하기위해서
경우의 수로 만들었습니다.
처음에 60개 건물을 한번에 예측하는데 써봤으나, 결국 60개의 개별 건물별 성능을 측정하면서 결국 combi 피쳐는 쓸모없어져서 나중에는 최종 사용할 속성에 포함하지 않습니다.

In [ ]:
train['cool_hot_combi'] = 0
test['cool_hot_combi'] = 0

for i in tqdm(range(len(train))):
  if (train['비전기냉방설비운영'].iloc[i] == 0) & (train['태양광보유'].iloc[i] == 1):
    train['cool_hot_combi'].iloc[i] = 1
    continue

  if (train['비전기냉방설비운영'].iloc[i] == 1) & (train['태양광보유'].iloc[i] == 0):
    train['cool_hot_combi'].iloc[i] = 2
    continue

  if (train['비전기냉방설비운영'].iloc[i] == 1) & (train['태양광보유'].iloc[i] == 1):
    train['cool_hot_combi'].iloc[i] = 3
    continue

for i in tqdm(range(len(test))):
  if (test['비전기냉방설비운영'].iloc[i] == 0) & (test['태양광보유'].iloc[i] == 1):
    test['cool_hot_combi'].iloc[i] = 1
    continue

  if (test['비전기냉방설비운영'].iloc[i] == 1) & (test['태양광보유'].iloc[i] == 0):
    test['cool_hot_combi'].iloc[i] = 2
    continue

  if (test['비전기냉방설비운영'].iloc[i] == 1) & (test['태양광보유'].iloc[i] == 1):
    test['cool_hot_combi'].iloc[i] = 3
    continue


  1%|▏         | 1621/122400 [00:00<00:07, 16201.98it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
100%|██████████| 10080/10080 [00:01<00:00, 5712.19it/s]


In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,비전기냉방설비운영,태양광보유,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,0,0,6,23,1,1,0,0,0,0
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,0.0,0.0,1,0,6,23,1,1,0,0,1,0
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,1.0,1.0,19,0,8,35,24,5,0,0,2035,3
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,1.0,1.0,20,0,8,35,24,5,0,0,2036,3
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,1.0,1.0,21,0,8,35,24,5,0,0,2037,3
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,1.0,1.0,22,0,8,35,24,5,0,0,2038,3


In [ ]:
train = train.drop('비전기냉방설비운영', axis=1)
train = train.drop('태양광보유', axis=1)

test = test.drop('비전기냉방설비운영', axis=1)
test = test.drop('태양광보유', axis=1)

In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,6,23,1,1,0,0,0,0
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,1,0,6,23,1,1,0,0,1,0
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,19,0,8,35,24,5,0,0,2035,3
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,20,0,8,35,24,5,0,0,2036,3
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,21,0,8,35,24,5,0,0,2037,3
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,22,0,8,35,24,5,0,0,2038,3


# 불쾌지수

https://dacon.io/competitions/official/235736/codeshare/2751?page=1&dtype=recent

In [ ]:
# 참고 : https://www.weatheri.co.kr/useful/useful04.php)
def Discomfort_Index(x):
    '''
    disscomfort index : 불쾌지수
    '''
    temp = x[0]
    humid = x[1]
    return 0.81 * temp +0.01 * humid * (0.99*temp - 14.3)+ 46.3

In [ ]:
train['Discomfort_Index']=train[['기온','습도']].apply(Discomfort_Index,axis=1)
test['Discomfort_Index']=test[['기온','습도']].apply(Discomfort_Index,axis=1)

# 체감온도 - 추가시 성능하락하여 제외 ( 6.4 -> 6.64 ) LGBM으로 재시도 ( 성능하락 제외 )

주로 겨울에 사용

체감온도(℃) = 13.12 + 0.6215 × T - 11.37 V0.16 + 0.3965 V0.16 × T

[T는 기온(℃), V는 풍속(km/h)]

https://m.blog.naver.com/sta_corp/220744977905

주로 겨울에 사용하고, 사용해봤을때 성능 하락이 있어서 제외했습니다.

In [ ]:
# def get_pow(series):
#     return math.pow(series, 0.16)

In [ ]:
# train['PerTemp'] = 13.12 + 0.6215*train['기온'] - 11.37*train['풍속'].apply(get_pow) + 0.3965*train['풍속'].apply(get_pow)*train['기온']

In [ ]:
# test['PerTemp'] = 13.12 + 0.6215*test['기온'] - 11.37*test['풍속'].apply(get_pow) + 0.3965*test['풍속'].apply(get_pow)*test['기온']

# 열지수

https://210life.tistory.com/entry/%EC%97%B4%EC%A7%80%EC%88%98-%EB%9E%80-%EA%B3%84%EC%82%B0%EC%9D%80-Heat-Index

heat index equation

HI = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH

where T is temperature **in degrees F** and RH is relative humidity in percent.  HI is the heat index expressed as an apparent temperature in degrees F.

T_f = (9/5) * T_c +32

여름에 사용하는 지수 중 하나로써, 파생변수로써 사용했습니다.

In [ ]:
def Heat_Index(x):
    temp = x[0]
    humid = x[1]
    '''
    열 지수
    '''
    T_f = (9/5)*temp + 32

    HI =  -42.379 + 2.04901523*T_f + 10.14333127*humid - (0.22475541 * T_f * humid) -  0.00683783*T_f*T_f - 0.05481717*humid*humid + .00122874*T_f*T_f*humid + 0.00085282*T_f*humid*humid - 0.00000199*T_f*T_f*humid*humid

    return HI

In [ ]:
train['Heat_Index']=train[['기온','습도']].apply(Heat_Index,axis=1)
test['Heat_Index']=test[['기온','습도']].apply(Heat_Index,axis=1)

## time + weekday str로 합친 카테고리를 만들면 어떨까 5.88 -> 5.92 성능하락 제외

In [ ]:
# train['time_weekday'] = train['time'].astype(str) + "_" + train['weekday'].astype(str)

In [ ]:
# test['time_weekday'] = test['time'].astype(str) + "_" + test['weekday'].astype(str)

# 31, 33 건물의 전력소비량 이상치를 대체해야함.

In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,6,23,1,1,0,0,0,0,63.43008,62.611766
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,1,0,6,23,1,1,0,0,1,0,63.56993,63.358359
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0,63.22775,63.383696
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0,62.54339,63.503185
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0,62.39760,62.747272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,19,0,8,35,24,5,0,0,2035,3,77.80896,86.078481
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,20,0,8,35,24,5,0,0,2036,3,77.44917,85.047329
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,21,0,8,35,24,5,0,0,2037,3,77.44917,85.047329
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,22,0,8,35,24,5,0,0,2038,3,77.52246,84.894919


In [ ]:
# 시간간격 257 의 값이 2285로써, 상당히 이상하게 나옴. 6200 -> 2285 -> 5810
train[(train['num']==31)&(train['시간간격']>255)&(train['시간간격']<259)]

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
61456,31,6200.064,31.3,3.6,44.0,0.0,1.0,16,3,6,24,11,2,0,0,256,2,78.99528,89.540322
61457,31,2285.280,31.4,3.2,44.0,0.0,1.0,17,3,6,24,11,2,0,0,257,2,79.11984,89.807078
61458,31,5810.400,30.5,3.2,45.0,0.0,1.0,18,3,6,24,11,2,0,0,258,2,78.15775,87.778109


In [ ]:
# 시간간격 257 의 값이 2285로써, 상당히 이상하게 나옴. 2800 -> 864 -> 2556
train[(train['num']==33)&(train['시간간격']>255)&(train['시간간격']<259)]

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
65536,33,2800.656,30.0,0.6,61.0,0.0,0.9,16,3,6,24,11,2,0,0,256,2,79.99400,91.460470
65537,33,884.304,30.6,1.3,59.0,0.0,0.8,17,3,6,24,11,2,0,0,257,2,80.52246,92.818450
65538,33,2556.144,30.6,0.8,57.0,0.0,1.0,18,3,6,24,11,2,0,0,258,2,80.20258,92.029621


In [ ]:
train.iloc[61457,1] = (6200.064 + 5810.400)/2
train.iloc[65537,1] = (2800.656 + 2556.144)/2

In [ ]:
train[(train['num']==31)&(train['시간간격']>255)&(train['시간간격']<259)]

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
61456,31,6200.064,31.3,3.6,44.0,0.0,1.0,16,3,6,24,11,2,0,0,256,2,78.99528,89.540322
61457,31,6005.232,31.4,3.2,44.0,0.0,1.0,17,3,6,24,11,2,0,0,257,2,79.11984,89.807078
61458,31,5810.400,30.5,3.2,45.0,0.0,1.0,18,3,6,24,11,2,0,0,258,2,78.15775,87.778109


In [ ]:
train[(train['num']==33)&(train['시간간격']>255)&(train['시간간격']<259)]

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
65536,33,2800.656,30.0,0.6,61.0,0.0,0.9,16,3,6,24,11,2,0,0,256,2,79.99400,91.460470
65537,33,2678.400,30.6,1.3,59.0,0.0,0.8,17,3,6,24,11,2,0,0,257,2,80.52246,92.818450
65538,33,2556.144,30.6,0.8,57.0,0.0,1.0,18,3,6,24,11,2,0,0,258,2,80.20258,92.029621


# CDH - LGBM에서 상승, H2O에서 하락 LGBM : 6점대에서 5.9로 진입.

https://dacon.io/competitions/official/235736/codeshare/2743?page=1&dtype=recent

i < 11, 5 이 구간에서 값 수정이 필요함. 

# 유사 컨셉 : CDHI ( HI cumm ), CDDI, CD풍, CD강, CD일조, ( 성능 하락 5.99 )


## 유사 컨셉 : 강수량, 일조량은 time interval 을 2,3으로..? ( 12h -> 6h : 5.89 -> 6.09 성능 하락, 24h : 5.89 -> 6.21  ) => 성능하락 ( 5.89 -> 5.99 )

In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,6,23,1,1,0,0,0,0,63.43008,62.611766
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,1,0,6,23,1,1,0,0,1,0,63.56993,63.358359
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0,63.22775,63.383696
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0,62.54339,63.503185
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0,62.39760,62.747272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,19,0,8,35,24,5,0,0,2035,3,77.80896,86.078481
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,20,0,8,35,24,5,0,0,2036,3,77.44917,85.047329
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,21,0,8,35,24,5,0,0,2037,3,77.44917,85.047329
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,22,0,8,35,24,5,0,0,2038,3,77.52246,84.894919


12시간 cdh를 할때 처음 0,,,11까지 적절히 보강하지 않으면 

당시 기온이 26도임을 뜻하는게 되어, 

다음날의 0,,,11 기온을 가져와서 보강했습니다. 

그게   np.sum(xs[(36-_j+1+i):36] 입니다.

In [ ]:
_j = 12
def CDH(xs):

    ys = []
    for i in range(len(xs)):
        if i < _j-1: # 
            ys.append(
                      np.sum(xs[:(i+1)]-26) + np.sum(xs[(36-_j+1+i):36]-26) 
                      ) # 여기다가, xs[:1]-26 * ( 5-(i)) 만큼 해줘야할것 같음. i=0, 5개의 값을 더해줘서 총 6개 유지. 
        else:
            ys.append(np.sum(xs[(i-_j+1):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,61,1):
    temp = train[train['num'] == num]
    # print(i,len(temp['기온'].values))
    cdh = CDH(temp['기온'].values)
    cdhs = np.concatenate([cdhs, cdh]) 
    # print(i,cdhs.shape)
train['CDH'] = cdhs

cdhs = np.array([])
for num in range(1,61,1):
    temp = test[test['num'] == num]
    # print(i,len(temp['기온'].values))
    cdh = CDH(temp['기온'].values)
    cdhs = np.concatenate([cdhs, cdh]) 
    # print(i,cdhs.shape)
test['CDH'] = cdhs

In [ ]:
# train[train['num']==1][['CDH','기온']].head(60)

# [전력 사용 추세선] - 지금은 시간간격 단변량인데, 다변량으로 바꿔보면 어떨까. 5.88 -> 6.1

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
train['전력사용추세'] = 0
test['전력사용추세'] = 0

for i in tqdm(range(60)):
  line_fitter = LinearRegression()
  # line_fitter.fit(train[train['num']==i+1][['기온',	'풍속',	'습도',	'강수량',	'일조', 'Discomfort_Index',	'Heat_Index','CDH','시간간격']].values.reshape(-1, 1), train[train['num']==i+1]['전력사용량'].values.reshape(-1, 1)) 
  line_fitter.fit(train[train['num']==i+1][['시간간격']].values.reshape(-1, 1), train[train['num']==i+1]['전력사용량'].values.reshape(-1, 1)) 

  train.loc[train['num']==i+1, '전력사용추세'] = line_fitter.predict(train[train['num']==i+1][['시간간격']].values.reshape(-1, 1))
  test.loc[test['num']==i+1, '전력사용추세'] = line_fitter.predict(test[test['num']==i+1][['시간간격']].values.reshape(-1, 1))

100%|██████████| 60/60 [00:00<00:00, 88.94it/s]


In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,6,23,1,1,0,0,0,0,63.43008,62.611766,193.9,8447.484058
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,1,0,6,23,1,1,0,0,1,0,63.56993,63.358359,195.4,8447.578015
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0,63.22775,63.383696,197.0,8447.671973
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0,62.54339,63.503185,198.4,8447.765930
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0,62.39760,62.747272,199.9,8447.859887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,19,0,8,35,24,5,0,0,2035,3,77.80896,86.078481,342.8,3668.845603
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,20,0,8,35,24,5,0,0,2036,3,77.44917,85.047329,344.3,3669.154883
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,21,0,8,35,24,5,0,0,2037,3,77.44917,85.047329,344.5,3669.464163
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,22,0,8,35,24,5,0,0,2038,3,77.52246,84.894919,343.3,3669.773442


# 1주 전 전력사용량



성능 향상이 있었습니다.

...

(추가할 아이디어 : 생각만하고 테스트는 안해봤습니다.)

%% 저전 차이 : 예측치를 기반으로 차이를 계산해보는게 어떨까?


In [ ]:
train_time_series = pd.DataFrame(data=train.values, columns = train.columns)
test_time_series = pd.DataFrame(data=test.values, columns = test.columns)

In [ ]:
# test의 발전량을 알 수 없어서 저전 계산이 불가능함.
# 저번주 발전량
# train_time_series['저전'] = 0
# test_time_series['저전'] = 0
# 저번주 전력추가시 성능 하락

# 저번주 기온 차이 -5
train_time_series['저기_차이'] = 0
test_time_series['저기_차이'] = 0

# 저번주 풍속 차이 -4
train_time_series['저풍_차이'] = 0
test_time_series['저풍_차이'] = 0

# 저번주 습도 차이 -3
train_time_series['저습_차이'] = 0
test_time_series['저습_차이'] = 0

# 저번주 강수량 차이 -2
train_time_series['저강_차이'] = 0
test_time_series['저강_차이'] = 0

# 저번주 일조 차이 -1
train_time_series['저일_차이'] = 0
test_time_series['저일_차이'] = 0

In [ ]:
train_time_series

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이
0,1.0,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,0.0,0.0,63.43008,62.611766,193.9,8447.484058,0,0,0,0,0
1,1.0,8135.640,17.7,2.9,91.0,0.3,0.0,1.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,63.56993,63.358359,195.4,8447.578015,0,0,0,0,0
2,1.0,8107.128,17.5,3.2,91.0,0.0,0.0,2.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,2.0,0.0,63.22775,63.383696,197.0,8447.671973,0,0,0,0,0
3,1.0,8048.808,17.1,3.2,91.0,0.0,0.0,3.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,3.0,0.0,62.54339,63.503185,198.4,8447.765930,0,0,0,0,0
4,1.0,8043.624,17.0,3.3,92.0,0.0,0.0,4.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,4.0,0.0,62.39760,62.747272,199.9,8447.859887,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60.0,4114.368,27.8,2.3,68.0,0.0,0.7,19.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2035.0,3.0,77.80896,86.078481,342.8,3668.845603,0,0,0,0,0
122396,60.0,3975.696,27.3,1.2,71.0,0.0,0.0,20.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2036.0,3.0,77.44917,85.047329,344.3,3669.154883,0,0,0,0,0
122397,60.0,3572.208,27.3,1.8,71.0,0.0,0.0,21.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2037.0,3.0,77.44917,85.047329,344.5,3669.464163,0,0,0,0,0
122398,60.0,3299.184,27.1,1.8,74.0,0.0,0.0,22.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2038.0,3.0,77.52246,84.894919,343.3,3669.773442,0,0,0,0,0


아래 코드는 인덱스 기반으로 계산했는데 하고나서 이렇게 코드를 작성한걸 상당히 후회했습니다.

가독성이 너무 안좋아서요. ㅠㅠ

In [ ]:
# 그러면 처음 일주일 데이터는 제외하고 새로운 칼럼에 전 주 발전량을 넣고, test에는 마지막 1주전 발젼량 ㄱㄱ
for i in tqdm(range(60)):
  # 122400 / 
  for j in range(int(2040-168)):
    # 저번주_전력 칼럼에는 실제 저번주 발전량 데이터가 들어간다.
    # 2040*i : num당 2040 개씩 존재해서 0, 2040, 4080 이렇게 인덱스가 증가함
    # j + 168 : 저번주_전력 칼럼의 값
    # j : 대입될, 실제 저번주 전력 값.
    # 좌측 : -6 : 저전_차이
    # 우측 : 1 : 이번주_전력 - 저번주_전력

    # # 저전
    # train_time_series.iloc[2040*i + j+168,-6] = train_time_series.iloc[2040*i + j,1]
    # 저기
    train_time_series.iloc[2040*i + j+168,-5] = train_time_series.iloc[2040*i + j+168,2] - train.iloc[2040*i + j,2]
    # 저풍
    train_time_series.iloc[2040*i + j+168,-4] = train_time_series.iloc[2040*i + j+168,3] - train.iloc[2040*i + j,3]
    train_time_series.iloc[2040*i + j+168,-3] = train_time_series.iloc[2040*i + j+168,4] - train.iloc[2040*i + j,4]
    train_time_series.iloc[2040*i + j+168,-2] = train_time_series.iloc[2040*i + j+168,5] - train.iloc[2040*i + j,5]
    train_time_series.iloc[2040*i + j+168,-1] = train_time_series.iloc[2040*i + j+168,6] - train.iloc[2040*i + j,6]

  for k in range(168):
    # 168*i : test의 num 당 인덱스가 0, 168, ... 이런식으로 시작
    # 1704*i : train의 num 당 1704 에서 시작함.

    # # 저전
    # test_time_series.iloc[168*i + k,-6] = train_time_series.iloc[2040*(i+1) -168 + k,1]

    # test의 발전량을 알 수 없어서 저전 차이 계산이 불가능함.
    # test의 1 : 기온, train 의 2 : 기온
    # 저기
    test_time_series.iloc[168*i + k,-5] = test_time_series.iloc[168*i + k,1] - train.iloc[2040*(i+1) -168 + k,2]
    # 저풍
    test_time_series.iloc[168*i + k,-4] = test_time_series.iloc[168*i + k,2] - train.iloc[2040*(i+1) -168 + k,3]
    test_time_series.iloc[168*i + k,-3] = test_time_series.iloc[168*i + k,3] - train.iloc[2040*(i+1) -168 + k,4]
    test_time_series.iloc[168*i + k,-2] = test_time_series.iloc[168*i + k,4] - train.iloc[2040*(i+1) -168 + k,5]
    test_time_series.iloc[168*i + k,-1] = test_time_series.iloc[168*i + k,5] - train.iloc[2040*(i+1) -168 + k,6]

100%|██████████| 60/60 [01:59<00:00,  1.99s/it]


In [ ]:
# test 의 첫 날 = 8/25 , 일주일전 : 8/18 
train_time_series[(train_time_series['num']==2)&(train_time_series['day']==18)].tail(24)

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이
3912,2.0,1089.612,26.3,0.0,81.0,0.0,0.0,0.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1872.0,2.0,77.10997,82.961021,344.6,1353.107530,0.3,-9.4,-12.0,0.0,0.0
3913,2.0,1084.752,26.1,0.0,79.0,0.0,0.0,1.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1873.0,2.0,76.55681,82.070431,338.8,1353.169927,-0.4,-9.3,-9.0,0.0,0.0
3914,2.0,1078.920,26.3,0.6,71.0,0.0,0.0,2.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1874.0,2.0,75.93627,82.016842,334.0,1353.232323,0.0,-7.2,-17.0,0.0,0.0
3915,2.0,1073.088,26.7,2.6,72.0,0.0,0.0,3.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1875.0,2.0,76.66276,83.300208,330.2,1353.294720,0.8,-4.3,-19.0,0.0,0.0
3916,2.0,1076.652,26.5,0.8,79.0,0.0,0.0,4.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1876.0,2.0,77.19365,83.483906,326.1,1353.357117,0.5,-6.1,-9.0,0.0,0.0
3917,2.0,1075.680,26.5,0.6,77.0,0.0,0.0,5.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1877.0,2.0,76.95495,83.253401,322.5,1353.419513,0.3,-5.8,-10.0,0.0,0.0
3918,2.0,1167.696,26.5,0.3,78.0,0.0,0.0,6.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1878.0,2.0,77.07430,83.368142,319.3,1353.481910,0.8,-4.6,-13.0,0.0,0.0
3919,2.0,1583.388,27.3,0.0,78.0,0.0,0.5,7.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1879.0,2.0,78.34006,86.355739,318.4,1353.544307,1.4,-4.3,-11.0,0.0,0.2
3920,2.0,1961.496,29.5,1.1,72.0,0.0,1.0,8.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1880.0,2.0,80.92660,93.713240,320.7,1353.606703,2.3,-5.1,-12.0,0.0,0.1
3921,2.0,2042.172,30.5,1.8,69.0,0.0,1.0,9.0,1.0,8.0,34.0,18.0,4.0,0.0,0.0,1881.0,2.0,81.97255,96.832887,324.7,1353.669100,2.5,-3.3,-15.0,0.0,0.0


In [ ]:
test_time_series[test_time_series['num']==2]

,num,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이
168,2.0,26.700000,0.900000,87.000000,0.0,0.000000,0.0,1.0,8.0,35.0,25.0,5.0,0.0,0.0,2040.0,2.0,78.482710,85.357654,338.000000,1363.590171,0.400000,0.900000,6.000000,0.0,0.000000
169,2.0,26.533333,0.900000,87.000000,0.0,0.000000,1.0,1.0,8.0,35.0,25.0,5.0,0.0,0.0,2041.0,2.0,78.204160,84.597145,336.333333,1363.652568,0.433333,0.900000,8.000000,0.0,0.000000
170,2.0,26.366667,0.900000,87.000000,0.0,0.000000,2.0,1.0,8.0,35.0,25.0,5.0,0.0,0.0,2042.0,2.0,77.925610,83.851937,334.700000,1363.714965,0.066667,0.300000,16.000000,0.0,0.000000
171,2.0,26.200000,0.900000,87.000000,0.0,0.000000,3.0,1.0,8.0,35.0,25.0,5.0,0.0,0.0,2043.0,2.0,77.647060,83.122029,333.100000,1363.777361,-0.500000,-1.700000,15.000000,0.0,0.000000
172,2.0,26.100000,0.800000,86.666667,0.0,0.000000,4.0,1.0,8.0,35.0,25.0,5.0,0.0,0.0,2044.0,2.0,77.441467,82.665233,331.400000,1363.839758,-0.400000,0.000000,7.666667,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
331,2.0,25.800000,3.800000,69.666667,0.0,1.133333,19.0,0.0,8.0,36.0,31.0,6.0,0.0,0.0,2203.0,2.0,75.029927,80.605422,328.066667,1373.760829,-2.100000,2.100000,-13.333333,0.0,0.533333
332,2.0,25.200000,3.600000,70.333333,0.0,0.566667,20.0,0.0,8.0,36.0,31.0,6.0,0.0,0.0,2204.0,2.0,74.201093,79.207720,326.500000,1373.823226,-2.200000,2.200000,-12.666667,0.0,0.566667
333,2.0,24.600000,3.400000,71.000000,0.0,0.000000,21.0,0.0,8.0,36.0,31.0,6.0,0.0,0.0,2205.0,2.0,73.364340,77.911545,323.600000,1373.885623,-2.500000,2.000000,-15.000000,0.0,0.000000
334,2.0,25.533333,2.633333,75.000000,0.0,0.000000,22.0,0.0,8.0,36.0,31.0,6.0,0.0,0.0,2206.0,2.0,75.215500,80.111014,321.866667,1373.948019,-1.366667,2.633333,-9.000000,0.0,0.000000


In [ ]:
train_time_series[train_time_series['시간간격']>=168][:1872]

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이
168,1.0,8584.056,20.6,2.0,81.0,0.0,0.0,0.0,0.0,6.0,24.0,8.0,2.0,0.0,0.0,168.0,0.0,67.92214,70.049675,297.5,8463.268878,3.0,-0.5,-11.0,-0.8,0.0
169,1.0,8582.112,19.9,2.6,85.0,0.0,0.0,1.0,0.0,6.0,24.0,8.0,2.0,0.0,0.0,169.0,0.0,67.00985,67.756599,291.3,8463.362836,2.2,-0.3,-6.0,-0.3,0.0
170,1.0,8567.856,19.8,1.1,85.0,0.0,0.0,2.0,0.0,6.0,24.0,8.0,2.0,0.0,0.0,170.0,0.0,66.84470,67.688126,283.7,8463.456793,2.3,-2.1,-6.0,0.0,0.0
171,1.0,8561.376,19.8,1.1,86.0,0.0,0.0,3.0,0.0,6.0,24.0,8.0,2.0,0.0,0.0,171.0,0.0,66.89772,67.184166,275.1,8463.550750,2.7,-2.1,-5.0,0.0,0.0
172,1.0,8548.416,20.0,0.9,86.0,0.0,0.0,4.0,0.0,6.0,24.0,8.0,2.0,0.0,0.0,172.0,0.0,67.23000,67.341475,266.2,8463.644707,3.0,-2.4,-6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,1.0,8714.952,29.4,3.4,66.0,0.0,0.2,19.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2035.0,0.0,79.88596,91.076702,355.7,8638.687089,-0.1,1.2,-7.0,0.0,-0.7
2036,1.0,8740.224,28.7,1.9,69.0,0.0,0.0,20.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2036.0,0.0,79.28497,89.447131,359.3,8638.781047,0.6,-0.9,-9.0,0.0,0.0
2037,1.0,8730.504,28.3,1.1,71.0,0.0,0.0,21.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2037.0,0.0,78.96207,88.533821,360.7,8638.875004,0.7,-1.2,-9.0,0.0,0.0
2038,1.0,8725.968,28.3,2.4,72.0,0.0,0.0,22.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2038.0,0.0,79.09924,88.806874,360.4,8638.968961,1.2,1.2,-9.0,0.0,0.0


# n(1~5) 시간 전 기온차이, 하루전, 이틀전, 3일전, 4일전, 5일전, 6일전 기온차이

한시간 전만 추가해봤으나 그다지 성능 향상이 없었습니다.

이틀전~6일전은 추가하니 성능 하락이 있었습니다.

In [ ]:
train_time_series['한전기_차이'] =0
test_time_series['한전기_차이'] =0

for i in tqdm(range(60)):
  globals()['train_'+str(i+1)] = train_time_series[train_time_series['num']==i+1].to_numpy()
  globals()['test_'+str(i+1)] = test_time_series[test_time_series['num']==i+1].to_numpy()
  for j in range(2040):
    if j>0:
      globals()['train_'+str(i+1)][j,-1]=globals()['train_'+str(i+1)][j,2] - globals()['train_'+str(i+1)][j-1,2]

  for k in range(168):
    if k==0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,1] - globals()['train_'+str(i+1)][-1,2]
    
    if k>0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,1] - globals()['test_'+str(i+1)][k-1,1]


train_all = pd.DataFrame()
test_all = pd.DataFrame()
for i in range(60):
  train_all = pd.concat([train_all, pd.DataFrame(globals()['train_'+str(i+1)]) ],axis=0)
  test_all = pd.concat([test_all, pd.DataFrame(globals()['test_'+str(i+1)]) ],axis=0)

train_time_series['한전기_차이'] = train_all[len(train_time_series.columns)-1].values
test_time_series['한전기_차이'] = test_all[len(test_time_series.columns)-1].values

100%|██████████| 60/60 [00:00<00:00, 105.86it/s]


# 한시간 전 HI, DI

In [ ]:
train_time_series['한전HI_차이'] =0
test_time_series['한전HI_차이'] =0

for i in tqdm(range(60)):
  globals()['train_'+str(i+1)] = train_time_series[train_time_series['num']==i+1].to_numpy()
  globals()['test_'+str(i+1)] = test_time_series[test_time_series['num']==i+1].to_numpy()
  for j in range(2040):
    if j>0:
      globals()['train_'+str(i+1)][j,-1]=globals()['train_'+str(i+1)][j,18] - globals()['train_'+str(i+1)][j-1,18]

  for k in range(168):
    if k==0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,17] - globals()['train_'+str(i+1)][-1,18]
    
    if k>0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,17] - globals()['test_'+str(i+1)][k-1,17]


train_all = pd.DataFrame()
test_all = pd.DataFrame()
for i in range(60):
  train_all = pd.concat([train_all, pd.DataFrame(globals()['train_'+str(i+1)]) ],axis=0)
  test_all = pd.concat([test_all, pd.DataFrame(globals()['test_'+str(i+1)]) ],axis=0)

train_time_series['한전HI_차이'] = train_all[len(train_time_series.columns)-1].values
test_time_series['한전HI_차이'] = test_all[len(test_time_series.columns)-1].values

100%|██████████| 60/60 [00:00<00:00, 120.09it/s]


In [ ]:
train_time_series

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이
0,1.0,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,0.0,0.0,63.43008,62.611766,193.9,8447.484058,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1.0,8135.640,17.7,2.9,91.0,0.3,0.0,1.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,63.56993,63.358359,195.4,8447.578015,0.0,0.0,0.0,0.0,0.0,0.1,0.746593
2,1.0,8107.128,17.5,3.2,91.0,0.0,0.0,2.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,2.0,0.0,63.22775,63.383696,197.0,8447.671973,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336
3,1.0,8048.808,17.1,3.2,91.0,0.0,0.0,3.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,3.0,0.0,62.54339,63.503185,198.4,8447.765930,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489
4,1.0,8043.624,17.0,3.3,92.0,0.0,0.0,4.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,4.0,0.0,62.39760,62.747272,199.9,8447.859887,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60.0,4114.368,27.8,2.3,68.0,0.0,0.7,19.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2035.0,3.0,77.80896,86.078481,342.8,3668.845603,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046
122396,60.0,3975.696,27.3,1.2,71.0,0.0,0.0,20.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2036.0,3.0,77.44917,85.047329,344.3,3669.154883,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152
122397,60.0,3572.208,27.3,1.8,71.0,0.0,0.0,21.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2037.0,3.0,77.44917,85.047329,344.5,3669.464163,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000
122398,60.0,3299.184,27.1,1.8,74.0,0.0,0.0,22.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2038.0,3.0,77.52246,84.894919,343.3,3669.773442,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410


In [ ]:
train_time_series['한전DI_차이'] =0
test_time_series['한전DI_차이'] =0

for i in tqdm(range(60)):
  globals()['train_'+str(i+1)] = train_time_series[train_time_series['num']==i+1].to_numpy()
  globals()['test_'+str(i+1)] = test_time_series[test_time_series['num']==i+1].to_numpy()
  for j in range(2040):
    if j>0:
      globals()['train_'+str(i+1)][j,-1]=globals()['train_'+str(i+1)][j,17] - globals()['train_'+str(i+1)][j-1,17]

  for k in range(168):
    if k==0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,16] - globals()['train_'+str(i+1)][-1,17]
    
    if k>0:
      globals()['test_'+str(i+1)][k,-1]=globals()['test_'+str(i+1)][k,16] - globals()['test_'+str(i+1)][k-1,16]


train_all = pd.DataFrame()
test_all = pd.DataFrame()
for i in range(60):
  train_all = pd.concat([train_all, pd.DataFrame(globals()['train_'+str(i+1)]) ],axis=0)
  test_all = pd.concat([test_all, pd.DataFrame(globals()['test_'+str(i+1)]) ],axis=0)

train_time_series['한전DI_차이'] = train_all[len(train_time_series.columns)-1].values
test_time_series['한전DI_차이'] = test_all[len(test_time_series.columns)-1].values

100%|██████████| 60/60 [00:00<00:00, 105.91it/s]


In [ ]:
train_time_series

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,1.0,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,0.0,0.0,63.43008,62.611766,193.9,8447.484058,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000
1,1.0,8135.640,17.7,2.9,91.0,0.3,0.0,1.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,63.56993,63.358359,195.4,8447.578015,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985
2,1.0,8107.128,17.5,3.2,91.0,0.0,0.0,2.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,2.0,0.0,63.22775,63.383696,197.0,8447.671973,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218
3,1.0,8048.808,17.1,3.2,91.0,0.0,0.0,3.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,3.0,0.0,62.54339,63.503185,198.4,8447.765930,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436
4,1.0,8043.624,17.0,3.3,92.0,0.0,0.0,4.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,4.0,0.0,62.39760,62.747272,199.9,8447.859887,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60.0,4114.368,27.8,2.3,68.0,0.0,0.7,19.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2035.0,3.0,77.80896,86.078481,342.8,3668.845603,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046,-0.75994
122396,60.0,3975.696,27.3,1.2,71.0,0.0,0.0,20.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2036.0,3.0,77.44917,85.047329,344.3,3669.154883,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152,-0.35979
122397,60.0,3572.208,27.3,1.8,71.0,0.0,0.0,21.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2037.0,3.0,77.44917,85.047329,344.5,3669.464163,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000,0.00000
122398,60.0,3299.184,27.1,1.8,74.0,0.0,0.0,22.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2038.0,3.0,77.52246,84.894919,343.3,3669.773442,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410,0.07329


# 12전 시간 차이도 추가해봤는데 그닥 성능 향상이 없었던것으로 기억합니다

# cat_cols 

LGBM 모델이 너무 오버피팅 되지 않게 각 일별 정규분포를 따르는 노이즈를 추가했습니다.

In [ ]:
train

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세
0,1,8179.056,17.6,2.5,92.0,0.8,0.0,0,0,6,23,1,1,0,0,0,0,63.43008,62.611766,193.9,8447.484058
1,1,8135.640,17.7,2.9,91.0,0.3,0.0,1,0,6,23,1,1,0,0,1,0,63.56993,63.358359,195.4,8447.578015
2,1,8107.128,17.5,3.2,91.0,0.0,0.0,2,0,6,23,1,1,0,0,2,0,63.22775,63.383696,197.0,8447.671973
3,1,8048.808,17.1,3.2,91.0,0.0,0.0,3,0,6,23,1,1,0,0,3,0,62.54339,63.503185,198.4,8447.765930
4,1,8043.624,17.0,3.3,92.0,0.0,0.0,4,0,6,23,1,1,0,0,4,0,62.39760,62.747272,199.9,8447.859887
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60,4114.368,27.8,2.3,68.0,0.0,0.7,19,0,8,35,24,5,0,0,2035,3,77.80896,86.078481,342.8,3668.845603
122396,60,3975.696,27.3,1.2,71.0,0.0,0.0,20,0,8,35,24,5,0,0,2036,3,77.44917,85.047329,344.3,3669.154883
122397,60,3572.208,27.3,1.8,71.0,0.0,0.0,21,0,8,35,24,5,0,0,2037,3,77.44917,85.047329,344.5,3669.464163
122398,60,3299.184,27.1,1.8,74.0,0.0,0.0,22,0,8,35,24,5,0,0,2038,3,77.52246,84.894919,343.3,3669.773442


In [ ]:
num_cols = ['기온', '풍속', '습도', '강수량', '일조','Heat_Index','전력사용추세','시간간격','Discomfort_Index','CDH',\
            '저기_차이',	'저풍_차이',	'저습_차이',	'저강_차이',	'저일_차이','한전기_차이','한전HI_차이','한전DI_차이'] # ,'CDrain','CDlight'] 성능하락 제외




cat_cols = ['time',	'weekday', 'month', 'week',	'day',	'weekend', 'week_of_month'] # cat cols 에서 num 제거


# train, train_time_series 의 전력사용량은 타겟으로 빠짐
print(len(num_cols)) # 17 -> 15 ( CDrain, CDlight ) -> 16 (nd_elec)
print(len(cat_cols)) # 8

18
7


In [ ]:
# train[num_cols]

train_time_series[num_cols]

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,17.6,2.5,92.0,0.8,0.0,62.611766,8447.484058,0.0,63.43008,193.9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000
1,17.7,2.9,91.0,0.3,0.0,63.358359,8447.578015,1.0,63.56993,195.4,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985
2,17.5,3.2,91.0,0.0,0.0,63.383696,8447.671973,2.0,63.22775,197.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218
3,17.1,3.2,91.0,0.0,0.0,63.503185,8447.765930,3.0,62.54339,198.4,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436
4,17.0,3.3,92.0,0.0,0.0,62.747272,8447.859887,4.0,62.39760,199.9,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,27.8,2.3,68.0,0.0,0.7,86.078481,3668.845603,2035.0,77.80896,342.8,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046,-0.75994
122396,27.3,1.2,71.0,0.0,0.0,85.047329,3669.154883,2036.0,77.44917,344.3,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152,-0.35979
122397,27.3,1.8,71.0,0.0,0.0,85.047329,3669.464163,2037.0,77.44917,344.5,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000,0.00000
122398,27.1,1.8,74.0,0.0,0.0,84.894919,3669.773442,2038.0,77.52246,343.3,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410,0.07329


In [ ]:
# len(train.columns)

len(train_time_series.columns) # 26 - 전력, num, cool hot combi : 23

29

# add noise

In [ ]:
# time series
train_time_series_num = train_time_series[num_cols]
train_time_series_num

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,17.6,2.5,92.0,0.8,0.0,62.611766,8447.484058,0.0,63.43008,193.9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000
1,17.7,2.9,91.0,0.3,0.0,63.358359,8447.578015,1.0,63.56993,195.4,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985
2,17.5,3.2,91.0,0.0,0.0,63.383696,8447.671973,2.0,63.22775,197.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218
3,17.1,3.2,91.0,0.0,0.0,63.503185,8447.765930,3.0,62.54339,198.4,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436
4,17.0,3.3,92.0,0.0,0.0,62.747272,8447.859887,4.0,62.39760,199.9,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,27.8,2.3,68.0,0.0,0.7,86.078481,3668.845603,2035.0,77.80896,342.8,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046,-0.75994
122396,27.3,1.2,71.0,0.0,0.0,85.047329,3669.154883,2036.0,77.44917,344.3,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152,-0.35979
122397,27.3,1.8,71.0,0.0,0.0,85.047329,3669.464163,2037.0,77.44917,344.5,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000,0.00000
122398,27.1,1.8,74.0,0.0,0.0,84.894919,3669.773442,2038.0,77.52246,343.3,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410,0.07329


In [ ]:
train_time_series_num = train_time_series[num_cols].to_numpy()

# new noise

In [ ]:
len(num_cols)

18

In [ ]:

for j,col in tqdm(enumerate(num_cols)):
  globals()['std_'+str(col)+'_list'] = []
  globals()['one_day_'+str(col)+'_list'] = []

  for i in (range(122400)):
    globals()['one_day_'+str(col)+'_list'].append(train_time_series_num[i,j])
    if i%24 == 23: 
      globals()['std_'+str(col)+'_list'].append(np.std(globals()['one_day_'+str(col)+'_list']))
      
      # 초기화
      globals()['one_day_'+str(col)+'_list'] = []

  # print(col,globals()['std_'+str(col)+'_list'])



noise_df = np.zeros((122400  ,len(num_cols))) # 실제 사용해야하는건 112320개 이지만 이것은 나중에 60개로 쪼개고 난뒤에 처리함.
noise_df2 = np.zeros((122400  ,len(num_cols))) # 저전 제외 : 16->15 , PerTemp 추가 : 15->16 . Per Temp 제외 : 15

j=0
np.random.seed(seed=43)
for i in tqdm(range(122400 )):
  for k,col in enumerate(num_cols):
    noise_df2[i,k] = np.random.normal(0,globals()['std_'+str(col)+'_list'][j])
  
  if i%24 == 23: 
    j = j+1

j=0
np.random.seed(seed=42)
for i in tqdm(range(122400 )):
  for k,col in enumerate(num_cols):
    noise_df[i,k] = np.random.normal(0,globals()['std_'+str(col)+'_list'][j])
  
  if i%24 == 23: 
    j = j+1

noise_df = pd.DataFrame(noise_df, columns = (train_time_series[num_cols]).columns)
noise_df2 = pd.DataFrame(noise_df2, columns = (train_time_series[num_cols]).columns)
noise_df

18it [00:05,  3.09it/s]
100%|██████████| 122400/122400 [00:10<00:00, 11559.52it/s]


,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,1.346553,-0.133770,13.498233,0.256364,-0.109407,-1.474845,1.027104,5.312326,-1.326682,14.403998,0.000000,0.000000,0.000000,0.0,0.000000,-0.474533,-1.214668,0.301771
1,-2.461582,-1.366393,30.545035,-0.038004,0.031552,-8.974588,-0.354061,0.767827,-3.252579,9.974110,0.000000,0.000000,0.000000,0.0,0.000000,-0.892637,0.986462,-1.172374
2,0.566213,-1.895966,-27.680226,0.033137,0.345043,1.079461,-0.075216,-2.084296,-4.178138,-19.110575,0.000000,0.000000,0.000000,0.0,0.000000,-0.324983,-0.811819,0.587391
3,2.794959,0.901006,-17.489817,-0.052048,0.154781,6.145027,-0.311650,-1.285166,-3.126379,-31.757144,0.000000,0.000000,0.000000,0.0,0.000000,-0.544438,0.433415,1.476973
4,-0.097122,1.513781,-54.597122,0.138347,0.040672,-1.883468,0.059680,-13.758323,-0.620768,9.480699,0.000000,0.000000,0.000000,0.0,0.000000,0.277444,-0.635331,0.492890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,2.082797,-2.398269,13.799843,0.000000,0.437454,3.617656,-0.963554,4.741065,-1.377033,-21.227842,1.668132,0.402801,-6.155629,0.0,0.379876,-0.926850,-0.153765,0.255339
122396,-0.194381,0.399171,9.691539,0.000000,-0.161374,2.475493,-1.939691,-7.977878,-1.094970,-23.803173,-0.590332,-0.147137,-2.203479,0.0,0.165246,-0.250161,-1.347746,0.579608
122397,-2.761590,0.113066,12.274712,0.000000,-0.092337,1.326732,-4.444143,0.793781,-0.297388,-42.310687,-0.719582,0.278704,11.365837,0.0,0.113877,-0.503544,1.231645,1.011211
122398,-3.040651,0.797914,8.014898,0.000000,-0.136181,-1.315561,-3.161735,-16.382004,0.616234,42.179078,0.563898,1.622830,10.409684,0.0,-0.018576,-0.015032,-0.879397,-0.618742


In [ ]:
noise_df2

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,0.697791,-0.878949,-7.888241,-0.090040,0.400929,-2.601579,0.324017,13.914974,3.568705,-11.660368,0.000000,0.000000,0.000000,0.0,0.000000,-0.104912,2.320770,-0.593889
1,-2.837899,-0.860698,0.292614,-0.027072,1.042120,-2.514058,0.035410,6.120471,-0.305141,14.750371,0.000000,0.000000,0.000000,0.0,0.000000,0.157966,-0.463711,1.593181
2,-5.549444,1.353829,-14.150932,0.257367,0.570605,6.393484,0.538607,15.687701,-1.681278,-15.431665,0.000000,0.000000,0.000000,0.0,0.000000,-0.901078,-1.292345,-0.765140
3,-4.028545,0.497416,17.751853,0.161369,-0.292720,1.939679,0.003386,4.786913,1.257131,2.396760,0.000000,0.000000,0.000000,0.0,0.000000,-1.096458,-0.385095,0.485785
4,5.663277,-0.979789,-0.499635,-0.161839,-0.043251,-1.409304,0.541704,6.743037,0.460106,-3.039558,0.000000,0.000000,0.000000,0.0,0.000000,-0.521321,-1.302897,-0.042977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,-2.342311,0.365753,-3.487357,0.000000,-0.077808,5.875917,-1.779089,-8.342571,0.760211,25.083597,-1.066492,0.587098,4.445902,0.0,-0.211824,-0.035961,-1.980783,0.831541
122396,0.786153,-0.080688,5.050748,0.000000,-0.163590,1.478936,4.543467,6.106788,0.914254,12.376768,0.865167,0.731942,-6.364241,0.0,-0.072018,-0.100507,-1.056233,0.475065
122397,-0.967170,0.339541,3.975220,0.000000,-0.555391,6.656676,-2.610380,2.004304,-0.307419,-14.987480,1.202716,1.479095,-2.069400,0.0,0.324399,0.169441,0.892185,1.026835
122398,-3.093809,-0.364725,-12.808809,0.000000,0.616077,-3.043069,3.797746,1.139751,-2.160839,-3.207942,-1.046990,-0.512490,5.794979,0.0,-0.204150,0.230969,-0.274382,-0.206691


# 노이즈 퍼센트

In [ ]:
# ------------- time series ----------------------

plus_noise_big = train_time_series[num_cols].values
plus_noise_big = pd.DataFrame(plus_noise_big)
plus_noise_big.columns = (train_time_series[num_cols]).columns
plus_noise_big = plus_noise_big + 0.08*pd.DataFrame(noise_df2) # 원래 더 크게 잡을 생각이었음

plus_noise_small = train_time_series[num_cols].values
plus_noise_small = pd.DataFrame(plus_noise_small)
plus_noise_small.columns = train_time_series[num_cols].columns
plus_noise_small = plus_noise_small + 0.08*pd.DataFrame(noise_df)

minus_noise_big = train_time_series[num_cols].values
minus_noise_big = pd.DataFrame(minus_noise_big)
minus_noise_big.columns = train_time_series[num_cols].columns
minus_noise_big = minus_noise_big - 0.08*pd.DataFrame(noise_df2) # 원래 더 크게 잡을 생각이었음

minus_noise_small = train_time_series[num_cols].values
minus_noise_small = pd.DataFrame(minus_noise_small)
minus_noise_small.columns = train_time_series[num_cols].columns
minus_noise_small = minus_noise_small - 0.08*pd.DataFrame(noise_df)

In [ ]:
train_time_series

,num,전력사용량,기온,풍속,습도,강수량,일조,time,weekday,month,week,day,week_of_month,holiday,weekend,시간간격,cool_hot_combi,Discomfort_Index,Heat_Index,CDH,전력사용추세,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이
0,1.0,8179.056,17.6,2.5,92.0,0.8,0.0,0.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,0.0,0.0,63.43008,62.611766,193.9,8447.484058,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000
1,1.0,8135.640,17.7,2.9,91.0,0.3,0.0,1.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,1.0,0.0,63.56993,63.358359,195.4,8447.578015,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985
2,1.0,8107.128,17.5,3.2,91.0,0.0,0.0,2.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,2.0,0.0,63.22775,63.383696,197.0,8447.671973,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218
3,1.0,8048.808,17.1,3.2,91.0,0.0,0.0,3.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,3.0,0.0,62.54339,63.503185,198.4,8447.765930,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436
4,1.0,8043.624,17.0,3.3,92.0,0.0,0.0,4.0,0.0,6.0,23.0,1.0,1.0,0.0,0.0,4.0,0.0,62.39760,62.747272,199.9,8447.859887,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,60.0,4114.368,27.8,2.3,68.0,0.0,0.7,19.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2035.0,3.0,77.80896,86.078481,342.8,3668.845603,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046,-0.75994
122396,60.0,3975.696,27.3,1.2,71.0,0.0,0.0,20.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2036.0,3.0,77.44917,85.047329,344.3,3669.154883,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152,-0.35979
122397,60.0,3572.208,27.3,1.8,71.0,0.0,0.0,21.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2037.0,3.0,77.44917,85.047329,344.5,3669.464163,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000,0.00000
122398,60.0,3299.184,27.1,1.8,74.0,0.0,0.0,22.0,0.0,8.0,35.0,24.0,5.0,0.0,0.0,2038.0,3.0,77.52246,84.894919,343.3,3669.773442,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410,0.07329


In [ ]:
# plus_noise_big = pd.concat([plus_noise_big,train[cat_cols]],axis=1)
# plus_noise_big

# ------------- time series -------------------------------

plus_noise_big = pd.concat([plus_noise_big,train_time_series[cat_cols]    ],axis=1)
plus_noise_big

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,17.655823,2.429684,91.368941,0.792797,0.032074,62.403640,8447.509979,1.113198,63.715576,192.967171,0.000000,0.000000,0.000000,0.0,0.000000,-0.008393,0.185662,-0.047511,0.0,0.0,6.0,23.0,1.0,0.0,1.0
1,17.472968,2.831144,91.023409,0.297834,0.083370,63.157235,8447.580848,1.489638,63.545519,196.580030,0.000000,0.000000,0.000000,0.0,0.000000,0.112637,0.709496,0.267304,1.0,0.0,6.0,23.0,1.0,0.0,1.0
2,17.056044,3.308306,89.867925,0.020589,0.045648,63.895174,8447.715061,3.255016,63.093248,195.765467,0.000000,0.000000,0.000000,0.0,0.000000,-0.272086,-0.078051,-0.403391,2.0,0.0,6.0,23.0,1.0,0.0,1.0
3,16.777716,3.239793,92.420148,0.012910,-0.023418,63.658359,8447.766201,3.382953,62.643960,198.591741,0.000000,0.000000,0.000000,0.0,0.000000,-0.487717,0.088681,-0.645497,3.0,0.0,6.0,23.0,1.0,0.0,1.0
4,17.453062,3.221617,91.960029,-0.012947,-0.003460,62.634527,8447.903224,4.539443,62.434408,199.656835,0.000000,0.000000,0.000000,0.0,0.000000,-0.141706,-0.860145,-0.149228,4.0,0.0,6.0,23.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,27.612615,2.329260,67.721011,0.000000,0.693775,86.548555,3668.703276,2034.332594,77.869777,344.806688,-0.285319,1.146968,-12.644328,0.0,-0.216946,-0.702877,-1.975509,-0.693417,19.0,0.0,8.0,35.0,24.0,0.0,5.0
122396,27.362892,1.193545,71.404060,0.000000,-0.013087,85.165644,3669.518360,2036.488543,77.522310,345.290141,0.569213,-0.841445,-15.509139,0.0,-0.005761,-0.508041,-1.115651,-0.321785,20.0,0.0,8.0,35.0,24.0,0.0,5.0
122397,27.222626,1.827163,71.318018,0.000000,-0.044431,85.579863,3669.255332,2037.160344,77.424576,343.301002,1.396217,-0.181672,-21.165552,0.0,0.025952,0.013555,0.071375,0.082147,21.0,0.0,8.0,35.0,24.0,0.0,5.0
122398,26.852495,1.770822,72.975295,0.000000,0.049286,84.651474,3670.077262,2038.091180,77.349593,343.043365,1.216241,0.759001,-12.536402,0.0,-0.016332,-0.181522,-0.174361,0.056755,22.0,0.0,8.0,35.0,24.0,0.0,5.0


In [ ]:
plus_noise_small = pd.concat([plus_noise_small,train[cat_cols]     ],axis=1)
minus_noise_big = pd.concat([minus_noise_big,train[cat_cols]       ],axis=1)
minus_noise_small = pd.concat([minus_noise_small,train[cat_cols]    ],axis=1)

minus_noise_small

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,17.492276,2.510702,90.920141,0.779491,0.008753,62.729754,8447.401890,-0.424986,63.536215,192.747680,0.000000,0.000000,0.000000,0.0,0.000000,0.037963,0.097173,-0.024142,0,0,6,23,1,0,1
1,17.896927,3.009311,88.556397,0.303040,-0.002524,64.076326,8447.606340,0.938574,63.830136,194.602071,0.000000,0.000000,0.000000,0.0,0.000000,0.171411,0.667676,0.233640,1,0,6,23,1,0,1
2,17.454703,3.351677,93.214418,-0.002651,-0.027603,63.297339,8447.677990,2.166744,63.562001,198.528846,0.000000,0.000000,0.000000,0.0,0.000000,-0.174001,0.090282,-0.389171,2,0,6,23,1,0,1
3,16.876403,3.127919,92.399185,0.004164,-0.012382,63.011582,8447.790862,3.102813,62.793500,200.940572,0.000000,0.000000,0.000000,0.0,0.000000,-0.356445,0.084816,-0.802518,3,0,6,23,1,0,1
4,17.007770,3.178898,96.367770,-0.011068,-0.003254,62.897949,8447.855113,5.100666,62.447261,199.141544,0.000000,0.000000,0.000000,0.0,0.000000,-0.122196,-0.705086,-0.185221,4,0,6,23,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,27.633376,2.491862,66.896013,0.000000,0.665004,85.789069,3668.922687,2034.620715,77.919123,344.498227,-0.333451,1.067776,-12.507550,0.0,-0.230390,-0.625852,-1.804745,-0.780367,19,0,8,35,24,0,5
122396,27.315550,1.168066,70.224677,0.000000,0.012910,84.849290,3669.310058,2036.638230,77.536768,346.204254,0.547227,-0.888229,-14.823722,0.0,-0.013220,-0.479987,-0.923332,-0.406159,20,0,8,35,24,0,5
122397,27.520927,1.790955,70.018023,0.000000,0.007387,84.941191,3669.819694,2036.936497,77.472961,347.884855,1.357567,-0.322296,-21.909267,0.0,-0.009110,0.040284,-0.098532,-0.080897,21,0,8,35,24,0,5
122398,27.343252,1.736167,73.358808,0.000000,0.010894,85.000164,3670.026381,2039.310560,77.473161,339.925674,1.254888,0.670174,-13.832775,0.0,0.001486,-0.198797,-0.082058,0.122789,22,0,8,35,24,0,5


# train_x, test_x 정의

In [ ]:
# --------- time series -------------------------------

normal = pd.concat([train_time_series[num_cols],train_time_series[cat_cols]    ],axis=1)
normal

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,17.6,2.5,92.0,0.8,0.0,62.611766,8447.484058,0.0,63.43008,193.9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,6.0,23.0,1.0,0.0,1.0
1,17.7,2.9,91.0,0.3,0.0,63.358359,8447.578015,1.0,63.56993,195.4,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985,1.0,0.0,6.0,23.0,1.0,0.0,1.0
2,17.5,3.2,91.0,0.0,0.0,63.383696,8447.671973,2.0,63.22775,197.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218,2.0,0.0,6.0,23.0,1.0,0.0,1.0
3,17.1,3.2,91.0,0.0,0.0,63.503185,8447.765930,3.0,62.54339,198.4,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436,3.0,0.0,6.0,23.0,1.0,0.0,1.0
4,17.0,3.3,92.0,0.0,0.0,62.747272,8447.859887,4.0,62.39760,199.9,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579,4.0,0.0,6.0,23.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122395,27.8,2.3,68.0,0.0,0.7,86.078481,3668.845603,2035.0,77.80896,342.8,-0.2,1.1,-13.0,0.0,-0.2,-0.7,-1.817046,-0.75994,19.0,0.0,8.0,35.0,24.0,0.0,5.0
122396,27.3,1.2,71.0,0.0,0.0,85.047329,3669.154883,2036.0,77.44917,344.3,0.5,-0.9,-15.0,0.0,0.0,-0.5,-1.031152,-0.35979,20.0,0.0,8.0,35.0,24.0,0.0,5.0
122397,27.3,1.8,71.0,0.0,0.0,85.047329,3669.464163,2037.0,77.44917,344.5,1.3,-0.3,-21.0,0.0,0.0,0.0,0.000000,0.00000,21.0,0.0,8.0,35.0,24.0,0.0,5.0
122398,27.1,1.8,74.0,0.0,0.0,84.894919,3669.773442,2038.0,77.52246,343.3,1.3,0.8,-13.0,0.0,0.0,-0.2,-0.152410,0.07329,22.0,0.0,8.0,35.0,24.0,0.0,5.0


In [ ]:
train_x = pd.concat([normal,plus_noise_small,minus_noise_small,plus_noise_big,minus_noise_big],axis=0, ignore_index=True )
train_x

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,17.600000,2.500000,92.000000,0.8,0.000000,62.611766,8447.484058,0.000000,63.430080,193.900000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,6.0,23.0,1.0,0.0,1.0
1,17.700000,2.900000,91.000000,0.3,0.000000,63.358359,8447.578015,1.000000,63.569930,195.400000,0.000000,0.000000,0.000000,0.0,0.000000,0.100000,0.746593,0.139850,1.0,0.0,6.0,23.0,1.0,0.0,1.0
2,17.500000,3.200000,91.000000,0.0,0.000000,63.383696,8447.671973,2.000000,63.227750,197.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.200000,0.025336,-0.342180,2.0,0.0,6.0,23.0,1.0,0.0,1.0
3,17.100000,3.200000,91.000000,0.0,0.000000,63.503185,8447.765930,3.000000,62.543390,198.400000,0.000000,0.000000,0.000000,0.0,0.000000,-0.400000,0.119489,-0.684360,3.0,0.0,6.0,23.0,1.0,0.0,1.0
4,17.000000,3.300000,92.000000,0.0,0.000000,62.747272,8447.859887,4.000000,62.397600,199.900000,0.000000,0.000000,0.000000,0.0,0.000000,-0.100000,-0.755913,-0.145790,4.0,0.0,6.0,23.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
611995,27.987385,2.270740,68.278989,0.0,0.706225,85.608408,3668.987930,2035.667406,77.748143,340.793312,-0.114681,1.053032,-13.355672,0.0,-0.183054,-0.697123,-1.658584,-0.826463,19.0,0.0,8.0,35.0,24.0,0.0,5.0
611996,27.237108,1.206455,70.595940,0.0,0.013087,84.929014,3668.791405,2035.511457,77.376030,343.309859,0.430787,-0.958555,-14.490861,0.0,0.005761,-0.491959,-0.946654,-0.397795,20.0,0.0,8.0,35.0,24.0,0.0,5.0
611997,27.377374,1.772837,70.681982,0.0,0.044431,84.514795,3669.672993,2036.839656,77.473764,345.698998,1.203783,-0.418328,-20.834448,0.0,-0.025952,-0.013555,-0.071375,-0.082147,21.0,0.0,8.0,35.0,24.0,0.0,5.0
611998,27.347505,1.829178,75.024705,0.0,-0.049286,85.138365,3669.469623,2037.908820,77.695327,343.556635,1.383759,0.840999,-13.463598,0.0,0.016332,-0.218478,-0.130459,0.089825,22.0,0.0,8.0,35.0,24.0,0.0,5.0


In [ ]:
# ----- time series ----

test_x = pd.concat([test_time_series[num_cols],test_time_series[cat_cols]   ],axis=1)

test_x

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,27.800000,1.500000,74.000000,0.0,0.000000,87.420247,8639.156876,2040.0,78.602280,342.7,1.600000,0.600000,-10.000000,0.0,0.000000,-0.300000,-0.891473,-0.327590,0.0,1.0,8.0,35.0,25.0,0.0,5.0
1,27.633333,1.366667,75.333333,0.0,0.000000,87.091155,8639.250833,2041.0,78.519273,342.0,1.933333,0.466667,-10.666667,0.0,0.000000,-0.166667,-0.329092,-0.083007,1.0,1.0,8.0,35.0,25.0,0.0,5.0
2,27.466667,1.233333,76.666667,0.0,0.000000,86.736660,8639.344790,2042.0,78.431867,341.5,2.066667,0.633333,-11.333333,0.0,0.000000,-0.166667,-0.354495,-0.087407,2.0,1.0,8.0,35.0,25.0,0.0,5.0
3,27.300000,1.100000,78.000000,0.0,0.000000,86.355739,8639.438747,2043.0,78.340060,341.2,2.300000,0.700000,-11.000000,0.0,0.000000,-0.166667,-0.380921,-0.091807,3.0,1.0,8.0,35.0,25.0,0.0,5.0
4,26.900000,1.166667,79.666667,0.0,0.000000,85.083042,8639.532705,2044.0,77.912697,340.7,2.100000,0.566667,-10.333333,0.0,0.000000,-0.400000,-1.272697,-0.427363,4.0,1.0,8.0,35.0,25.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10075,28.633333,3.566667,66.000000,0.0,0.533333,88.345089,3720.804610,2203.0,78.764020,334.4,0.833333,1.266667,-2.000000,0.0,-0.166667,-0.366667,-0.967477,-0.392480,19.0,0.0,8.0,36.0,31.0,0.0,6.0
10076,28.266667,3.833333,67.000000,0.0,0.266667,87.374853,3721.113890,2204.0,78.364280,337.4,0.966667,2.633333,-4.000000,0.0,0.266667,-0.366667,-0.970236,-0.399740,20.0,0.0,8.0,36.0,31.0,0.0,6.0
10077,27.900000,4.100000,68.000000,0.0,0.000000,86.402312,3721.423169,2205.0,77.957280,339.7,0.600000,2.300000,-3.000000,0.0,0.000000,-0.366667,-0.972541,-0.407000,21.0,0.0,8.0,36.0,31.0,0.0,6.0
10078,27.900000,4.100000,68.000000,0.0,0.000000,86.402312,3721.732449,2206.0,77.957280,341.3,0.800000,2.300000,-6.000000,0.0,0.000000,0.000000,0.000000,0.000000,22.0,0.0,8.0,36.0,31.0,0.0,6.0


In [ ]:
target = pd.concat( [train['전력사용량'],train['전력사용량'],train['전력사용량'],train['전력사용량'],train['전력사용량']],axis=0,ignore_index=True)
target

0         8179.056
1         8135.640
2         8107.128
3         8048.808
4         8043.624
            ...   
611995    4114.368
611996    3975.696
611997    3572.208
611998    3299.184
611999    3204.576
Name: 전력사용량, Length: 612000, dtype: float64

# 60개 건물별 데이터로 쪼개기

In [ ]:
train_x[:2040]

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,17.6,2.5,92.0,0.8,0.0,62.611766,8447.484058,0.0,63.43008,193.9,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,6.0,23.0,1.0,0.0,1.0
1,17.7,2.9,91.0,0.3,0.0,63.358359,8447.578015,1.0,63.56993,195.4,0.0,0.0,0.0,0.0,0.0,0.1,0.746593,0.13985,1.0,0.0,6.0,23.0,1.0,0.0,1.0
2,17.5,3.2,91.0,0.0,0.0,63.383696,8447.671973,2.0,63.22775,197.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.025336,-0.34218,2.0,0.0,6.0,23.0,1.0,0.0,1.0
3,17.1,3.2,91.0,0.0,0.0,63.503185,8447.765930,3.0,62.54339,198.4,0.0,0.0,0.0,0.0,0.0,-0.4,0.119489,-0.68436,3.0,0.0,6.0,23.0,1.0,0.0,1.0
4,17.0,3.3,92.0,0.0,0.0,62.747272,8447.859887,4.0,62.39760,199.9,0.0,0.0,0.0,0.0,0.0,-0.1,-0.755913,-0.14579,4.0,0.0,6.0,23.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,29.4,3.4,66.0,0.0,0.2,91.076702,8638.687089,2035.0,79.88596,355.7,-0.1,1.2,-7.0,0.0,-0.7,-0.8,-1.483584,-0.54680,19.0,0.0,8.0,35.0,24.0,0.0,5.0
2036,28.7,1.9,69.0,0.0,0.0,89.447131,8638.781047,2036.0,79.28497,359.3,0.6,-0.9,-9.0,0.0,0.0,-0.7,-1.629571,-0.60099,20.0,0.0,8.0,35.0,24.0,0.0,5.0
2037,28.3,1.1,71.0,0.0,0.0,88.533821,8638.875004,2037.0,78.96207,360.7,0.7,-1.2,-9.0,0.0,0.0,-0.4,-0.913309,-0.32290,21.0,0.0,8.0,35.0,24.0,0.0,5.0
2038,28.3,2.4,72.0,0.0,0.0,88.806874,8638.968961,2038.0,79.09924,360.4,1.2,1.2,-9.0,0.0,0.0,0.0,0.273052,0.13717,22.0,0.0,8.0,35.0,24.0,0.0,5.0


In [ ]:
# # https://congcoding.tistory.com/55

# ---------------- time_series -------------------------------------
for i in tqdm(range(60)): # 노이즈로 데이터량 증가
  for j in range(5):
    
    if (j < 1):
      # time_series인 경우, 첫 주 데이터는 제거해야함. 
      # 즉, train_x 시작점이 168 부터임.
      globals()['train_'+str(i+1)] = train_x[168+2040*i+(j*122400):2040*(i+1)+(j*122400)]


      # time_series인 경우,  : 좌측 2040 대신 2040 + 168 로 대체해야함. 
      globals()['target_'+str(i+1)] = target[2040*i+(j*122400)+168:2040*(i+1)+(j*122400)]

      globals()['test_'+str(i+1)] = test_x[168*i+(j*10080):168*(i+1)+(j*10080)]

    if (j >= 1):
      # time_series인 경우, 첫 주 데이터는 제거해야함. 
      # 즉, train_x 시작점이 168 부터임.
      globals()['train_'+str(i+1)] = pd.concat([globals()['train_'+str(i+1)] , train_x[168+2040*i+(j*122400):2040*(i+1)+(j*122400)]],axis=0)
      
      # time_series인 경우,  : 좌측 2040 대신 2040 + 168 로 대체해야함. 
      globals()['target_'+str(i+1)] = pd.concat([globals()['target_'+str(i+1)], target[2040*i+(j*122400)+168:2040*(i+1)+(j*122400)]], axis=0)

      globals()['test_'+str(i+1)] = pd.concat([globals()['test_'+str(i+1)] , test_x[168*i+(j*10080):168*(i+1)+(j*10080)]],axis=0)

# 위 cat_cols 에서 num 빼서 아래 코드에서 num 열을 드랍할 이유는 없음
  # globals()['train_'+str(i+1)] = globals()['train_'+str(i+1)].drop(['num'],axis=1)
  # globals()['test_'+str(i+1)] = globals()['test_'+str(i+1)].drop(['num'],axis=1)

100%|██████████| 60/60 [00:00<00:00, 75.11it/s]


In [ ]:
train_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
168,20.600000,2.000000,81.000000,0.0,0.000000,70.049675,8463.268878,168.000000,67.922140,297.500000,3.000000,-0.500000,-11.000000,-0.8,0.000000,-0.400000,-0.719894,-0.579860,0.0,0.0,6.0,24.0,8.0,0.0,2.0
169,19.900000,2.600000,85.000000,0.0,0.000000,67.756599,8463.362836,169.000000,67.009850,291.300000,2.200000,-0.300000,-6.000000,-0.3,0.000000,-0.700000,-2.293077,-0.912290,1.0,0.0,6.0,24.0,8.0,0.0,2.0
170,19.800000,1.100000,85.000000,0.0,0.000000,67.688126,8463.456793,170.000000,66.844700,283.700000,2.300000,-2.100000,-6.000000,0.0,0.000000,-0.100000,-0.068472,-0.165150,2.0,0.0,6.0,24.0,8.0,0.0,2.0
171,19.800000,1.100000,86.000000,0.0,0.000000,67.184166,8463.550750,171.000000,66.897720,275.100000,2.700000,-2.100000,-5.000000,0.0,0.000000,0.000000,-0.503960,0.053020,3.0,0.0,6.0,24.0,8.0,0.0,2.0
172,20.000000,0.900000,86.000000,0.0,0.000000,67.341475,8463.644707,172.000000,67.230000,266.200000,3.000000,-2.400000,-6.000000,0.0,0.000000,0.200000,0.157309,0.332280,4.0,0.0,6.0,24.0,8.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491635,28.988333,3.327847,65.161954,0.0,0.145371,91.775049,8638.685954,2034.671671,80.063661,355.338804,-0.119419,1.117017,-7.306965,0.0,-0.728692,-0.805301,-1.465807,-0.504892,19.0,0.0,8.0,35.0,24.0,0.0,5.0
491636,28.400673,1.942743,68.067387,0.0,-0.002668,88.265920,8638.775265,2035.163658,79.266210,358.280685,0.673200,-0.850040,-9.172332,0.0,0.025483,-0.698516,-1.709370,-0.543445,20.0,0.0,8.0,35.0,24.0,0.0,5.0
491637,27.986975,1.177698,69.893644,0.0,-0.036543,89.094764,8638.884358,2036.086502,78.835585,360.943888,0.690436,-1.249660,-9.191811,0.0,-0.004576,-0.321157,-1.325957,-0.351429,21.0,0.0,8.0,35.0,24.0,0.0,5.0
491638,28.378056,2.431350,70.068172,0.0,-0.002399,89.171682,8638.907881,2038.677244,78.972997,361.672082,1.190175,1.213366,-9.380995,0.0,-0.003852,-0.019209,0.049150,0.158121,22.0,0.0,8.0,35.0,24.0,0.0,5.0


In [ ]:
test_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month
0,27.800000,1.500000,74.000000,0.0,0.0,87.420247,8639.156876,2040.0,78.602280,342.700000,1.600000,0.600000,-10.000000,0.0,0.0,-0.300000,-0.891473,-0.327590,0.0,1.0,8.0,35.0,25.0,0.0,5.0
1,27.633333,1.366667,75.333333,0.0,0.0,87.091155,8639.250833,2041.0,78.519273,342.000000,1.933333,0.466667,-10.666667,0.0,0.0,-0.166667,-0.329092,-0.083007,1.0,1.0,8.0,35.0,25.0,0.0,5.0
2,27.466667,1.233333,76.666667,0.0,0.0,86.736660,8639.344790,2042.0,78.431867,341.500000,2.066667,0.633333,-11.333333,0.0,0.0,-0.166667,-0.354495,-0.087407,2.0,1.0,8.0,35.0,25.0,0.0,5.0
3,27.300000,1.100000,78.000000,0.0,0.0,86.355739,8639.438747,2043.0,78.340060,341.200000,2.300000,0.700000,-11.000000,0.0,0.0,-0.166667,-0.380921,-0.091807,3.0,1.0,8.0,35.0,25.0,0.0,5.0
4,26.900000,1.166667,79.666667,0.0,0.0,85.083042,8639.532705,2044.0,77.912697,340.700000,2.100000,0.566667,-10.333333,0.0,0.0,-0.400000,-1.272697,-0.427363,4.0,1.0,8.0,35.0,25.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,28.233333,2.800000,72.666667,0.0,0.8,88.733952,8654.471910,2203.0,79.088727,330.133333,-1.166667,-0.600000,6.666667,0.0,0.6,-0.566667,-2.036369,-0.771913,19.0,0.0,8.0,36.0,31.0,0.0,6.0
164,27.666667,2.600000,73.333333,0.0,0.4,86.774660,8654.565867,2204.0,78.309333,332.900000,-1.033333,0.700000,4.333333,0.0,0.4,-0.566667,-1.959293,-0.779393,20.0,0.0,8.0,36.0,31.0,0.0,6.0
165,27.100000,2.400000,74.000000,0.0,0.0,84.894919,8654.659824,2205.0,77.522460,334.600000,-1.200000,1.300000,3.000000,0.0,0.0,-0.566667,-1.879740,-0.786873,21.0,0.0,8.0,36.0,31.0,0.0,6.0
166,26.966667,1.900000,78.333333,0.0,0.0,85.126196,8654.753781,2206.0,77.853983,335.633333,-1.333333,-0.500000,6.333333,0.0,0.0,-0.133333,0.231277,0.331523,22.0,0.0,8.0,36.0,31.0,0.0,6.0


In [ ]:
target_1

168       8584.056
169       8582.112
170       8567.856
171       8561.376
172       8548.416
            ...   
491635    8714.952
491636    8740.224
491637    8730.504
491638    8725.968
491639    8705.232
Name: 전력사용량, Length: 9360, dtype: float64

In [ ]:
print(len(num_cols)) # 16
print(len(cat_cols)) # 8

18
7


# AutoML data prepare

In [ ]:
import automl_alex
from automl_alex import DataPrepare
from automl_alex import CrossValidation

In [ ]:
cat_cols

['time', 'weekday', 'month', 'week', 'day', 'weekend', 'week_of_month']

num_generate 를 쓰면 코랩(pro) 세션이 죽어버렸고,

num_denoise AE 는 썼을때 성능이 하락했습니다.

카테고리 인코더는 솔직히 Helmert, OneHot, Count, Sum 정도만 써도 될 것 같습니다.

그리고 타겟 인코더는 에러가 나던데, 다음엔 직접 Catboost 인코더 등을 써보면 좋지 않을까 싶었습니다.

카테고리 인코더별 성능을 측정해놓은 캐글 글도 있으니 참고해주시면 좋을것 같습니다. 

https://www.kaggle.com/subinium/11-categorical-encoders-and-benchmark

In [ ]:
%%time
de = DataPrepare(clean_and_encod_data=True,
                cat_encoder_names=['HelmertEncoder','OneHotEncoder','HashingEncoder','SumEncoder','BackwardDifferenceEncoder','CountEncoder'], # Encoders list for Generator cat encodet features
                # target_encoders_names= ["TargetEncoder","JamesSteinEncoder","CatBoostEncoder"],
                clean_nan=False, # fillnan
                clean_outliers=True, # method='IQR', threshold=2,
                drop_invariant=False, # drop invariant features (data.nunique < 2)
                num_generator_features=False, # Generator interaction Num Features
                # num_denoising_autoencoder=False, # denoising_autoencoder if num features > 2
                normalization=False, # normalization data (StandardScaler)
                cat_features=cat_cols, # DataPrepare can auto detect categorical features
                random_state=42,
                verbose=1) # 3 하니 렉걸림

for i in range(60):
  globals()['de_train_'+str(i+1)] = de.fit_transform(globals()['train_'+str(i+1)])
  globals()['de_test_'+str(i+1)] = de.transform(globals()['test_'+str(i+1)])

CPU times: user 1min 36s, sys: 3.43 s, total: 1min 39s
Wall time: 1min 39s


In [ ]:
de_train_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month,HelmertEncoder_time_0,HelmertEncoder_time_1,HelmertEncoder_time_2,HelmertEncoder_time_3,HelmertEncoder_time_4,HelmertEncoder_time_5,HelmertEncoder_time_6,HelmertEncoder_time_7,HelmertEncoder_time_8,HelmertEncoder_time_9,HelmertEncoder_time_10,HelmertEncoder_time_11,HelmertEncoder_time_12,HelmertEncoder_time_13,HelmertEncoder_time_14,...,BackwardDifferenceEncoder_day_25,BackwardDifferenceEncoder_day_26,BackwardDifferenceEncoder_day_27,BackwardDifferenceEncoder_day_28,BackwardDifferenceEncoder_day_29,BackwardDifferenceEncoder_weekend_0,BackwardDifferenceEncoder_week_of_month_0,BackwardDifferenceEncoder_week_of_month_1,BackwardDifferenceEncoder_week_of_month_2,BackwardDifferenceEncoder_week_of_month_3,CountEncoder_weekday,CountEncoder_month,CountEncoder_week,CountEncoder_day,CountEncoder_weekend,CountEncoder_week_of_month,week_of_month_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,시간간격_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,한전기_차이_Is_Outliers_IQR,한전DI_차이_Is_Outliers_IQR,습도_Is_Outliers_IQR,기온_Is_Outliers_IQR,week_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,weekday_Is_Outliers_IQR,CDH_Is_Outliers_IQR,한전HI_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,일조_Is_Outliers_IQR,강수량_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,풍속_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR
0,20.600000,2.000000,81.000000,0.0,0.000000,70.049675,8463.268878,168.000000,67.922140,297.500000,3.000000,-0.500000,-11.000000,-0.092702,0.000000,-0.400000,-0.719894,-0.579860,0.0,0.0,6.0,24.0,8.0,0.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440,2760,840,360,6720,2520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19.900000,2.600000,85.000000,0.0,0.000000,67.756599,8463.362836,169.000000,67.009850,291.300000,2.200000,-0.300000,-6.000000,-0.092702,0.000000,-0.700000,-2.293077,-0.912290,1.0,0.0,6.0,24.0,8.0,0.0,2.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440,2760,840,360,6720,2520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19.800000,1.100000,85.000000,0.0,0.000000,67.688126,8463.456793,170.000000,66.844700,283.700000,2.300000,-2.100000,-6.000000,0.000000,0.000000,-0.100000,-0.068472,-0.165150,2.0,0.0,6.0,24.0,8.0,0.0,2.0,0.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440,2760,840,360,6720,2520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19.800000,1.100000,86.000000,0.0,0.000000,67.184166,8463.550750,171.000000,66.897720,275.100000,2.700000,-2.100000,-5.000000,0.000000,0.000000,0.000000,-0.503960,0.053020,3.0,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440,2760,840,360,6720,2520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,20.000000,0.900000,86.000000,0.0,0.000000,67.341475,8463.644707,172.000000,67.230000,266.200000,3.000000,-2.400000,-6.000000,0.000000,0.000000,0.200000,0.157309,0.332280,4.0,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,0.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440,2760,840,360,6720,2520,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [ ]:
target_1

168       8584.056
169       8582.112
170       8567.856
171       8561.376
172       8548.416
            ...   
491635    8714.952
491636    8740.224
491637    8730.504
491638    8725.968
491639    8705.232
Name: 전력사용량, Length: 9360, dtype: float64

카테고리 별 오버피팅이 발생하는걸 방지하기 위해서 2시간 사이의 중간 값을 추가해줬습니다.

어떤 식이냐면

원핫을 했을때 월요일 밤 11시 -> 화요일 새벽 0시로 갈때

... | 시간23 | 시간 0 | 월요일원핫 | 화요일원핫 |

... |   1    |    0   |    1       |         0  |

... |   0    |    1   |    0       |         1  |

이라면 새로운 행으로

... |  0.5  | 0.5     |   0.5      |     0.5    |

이런식입니다.

그다지 효과적이진 않았던것으로 기억합니다.

# 두 날짜간 interpolate

In [ ]:
%%time
for i in range(60):
  globals()['de_train_n_target_'+str(i+1)] = pd.concat([globals()['de_train_'+str(i+1)],globals()['target_'+str(i+1)].reset_index(drop=True)], axis=1)

for j in range(60):
  globals()['avg_2rows_'+str(j+1)] = np.zeros((9359,len(de_train_1.columns)+1))
  for i in tqdm(range(1,9360)):
    globals()['avg_2rows_'+str(j+1)][i-1,:] = (globals()['de_train_n_target_'+str(j+1)].iloc[i,:].values + globals()['de_train_n_target_'+str(j+1)].iloc[i-1,:].values)/2

  
  globals()['avg_2rows_'+str(j+1)] = pd.DataFrame(globals()['avg_2rows_'+str(j+1)], columns=de_train_n_target_1.columns)
  globals()['avg_target_'+str(j+1)] = globals()['avg_2rows_'+str(j+1)]['전력사용량']
  globals()['avg_2rows_'+str(j+1)] = globals()['avg_2rows_'+str(j+1)].drop(['전력사용량'],axis=1)

for j in tqdm(range(60)):
  globals()['de_train_'+str(j+1)] = pd.concat([globals()['avg_2rows_'+str(j+1)],globals()['de_train_'+str(j+1)]],axis=0)
  globals()['target_'+str(j+1)] = pd.concat([globals()['avg_target_'+str(j+1)],globals()['target_'+str(j+1)]],axis=0)

100%|██████████| 60/60 [00:04<00:00, 12.70it/s]

CPU times: user 7min 51s, sys: 19 s, total: 8min 10s
Wall time: 7min 45s


In [ ]:
avg_2rows_3

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month,HelmertEncoder_time_0,HelmertEncoder_time_1,HelmertEncoder_time_2,HelmertEncoder_time_3,HelmertEncoder_time_4,HelmertEncoder_time_5,HelmertEncoder_time_6,HelmertEncoder_time_7,HelmertEncoder_time_8,HelmertEncoder_time_9,HelmertEncoder_time_10,HelmertEncoder_time_11,HelmertEncoder_time_12,HelmertEncoder_time_13,HelmertEncoder_time_14,...,BackwardDifferenceEncoder_day_25,BackwardDifferenceEncoder_day_26,BackwardDifferenceEncoder_day_27,BackwardDifferenceEncoder_day_28,BackwardDifferenceEncoder_day_29,BackwardDifferenceEncoder_weekend_0,BackwardDifferenceEncoder_week_of_month_0,BackwardDifferenceEncoder_week_of_month_1,BackwardDifferenceEncoder_week_of_month_2,BackwardDifferenceEncoder_week_of_month_3,CountEncoder_weekday,CountEncoder_month,CountEncoder_week,CountEncoder_day,CountEncoder_weekend,CountEncoder_week_of_month,week_of_month_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,시간간격_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,한전기_차이_Is_Outliers_IQR,한전DI_차이_Is_Outliers_IQR,습도_Is_Outliers_IQR,기온_Is_Outliers_IQR,week_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,weekday_Is_Outliers_IQR,CDH_Is_Outliers_IQR,한전HI_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,일조_Is_Outliers_IQR,강수량_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,풍속_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR
0,18.250000,1.800000,99.000000,0.0,0.000000,57.309270,3250.694984,168.500000,64.812820,275.250000,1.150000,-0.350000,-0.500000,-0.069990,0.000000,-0.200000,-1.398432,-0.296805,0.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,18.550000,1.750000,99.000000,0.0,0.000000,57.606715,3250.836769,169.500000,65.346880,267.900000,1.350000,-1.250000,0.500000,-0.034995,0.000000,0.300000,0.297445,0.534060,1.5,0.0,6.0,24.0,8.0,0.0,2.0,0.5,0.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,18.950000,1.100000,97.000000,0.0,0.000000,59.587500,3250.978553,170.500000,65.974700,259.850000,2.000000,-2.300000,-1.500000,0.000000,0.000000,0.400000,1.980785,0.627820,2.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.350000,0.750000,95.000000,0.0,0.000000,61.463684,3251.120338,171.500000,66.584700,251.450000,2.550000,-2.550000,-3.500000,0.000000,0.000000,0.400000,1.876184,0.610000,3.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,1.5,1.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,19.550000,1.050000,94.000000,0.0,0.000000,62.372126,3251.262123,172.500000,66.886730,243.700000,2.800000,-2.500000,-4.500000,0.000000,0.000000,0.200000,0.908442,0.302030,4.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...

In [ ]:
avg_2rows_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month,HelmertEncoder_time_0,HelmertEncoder_time_1,HelmertEncoder_time_2,HelmertEncoder_time_3,HelmertEncoder_time_4,HelmertEncoder_time_5,HelmertEncoder_time_6,HelmertEncoder_time_7,HelmertEncoder_time_8,HelmertEncoder_time_9,HelmertEncoder_time_10,HelmertEncoder_time_11,HelmertEncoder_time_12,HelmertEncoder_time_13,HelmertEncoder_time_14,...,BackwardDifferenceEncoder_day_25,BackwardDifferenceEncoder_day_26,BackwardDifferenceEncoder_day_27,BackwardDifferenceEncoder_day_28,BackwardDifferenceEncoder_day_29,BackwardDifferenceEncoder_weekend_0,BackwardDifferenceEncoder_week_of_month_0,BackwardDifferenceEncoder_week_of_month_1,BackwardDifferenceEncoder_week_of_month_2,BackwardDifferenceEncoder_week_of_month_3,CountEncoder_weekday,CountEncoder_month,CountEncoder_week,CountEncoder_day,CountEncoder_weekend,CountEncoder_week_of_month,week_of_month_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,시간간격_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,한전기_차이_Is_Outliers_IQR,한전DI_차이_Is_Outliers_IQR,습도_Is_Outliers_IQR,기온_Is_Outliers_IQR,week_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,weekday_Is_Outliers_IQR,CDH_Is_Outliers_IQR,한전HI_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,일조_Is_Outliers_IQR,강수량_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,풍속_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR
0,20.250000,2.300000,83.000000,0.0,0.000000,68.903137,8463.315857,168.500000,67.465995,294.400000,2.600000,-0.400000,-8.500000,-0.092702,0.000000,-0.550000,-1.506485,-0.746075,0.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.850000,1.850000,85.000000,0.0,0.000000,67.722362,8463.409814,169.500000,66.927275,287.500000,2.250000,-1.200000,-6.000000,-0.046351,0.000000,-0.400000,-1.180775,-0.538720,1.5,0.0,6.0,24.0,8.0,0.0,2.0,0.5,0.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19.800000,1.100000,85.500000,0.0,0.000000,67.436146,8463.503772,170.500000,66.871210,279.400000,2.500000,-2.100000,-5.500000,0.000000,0.000000,-0.050000,-0.286216,-0.056065,2.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.900000,1.000000,86.000000,0.0,0.000000,67.262821,8463.597729,171.500000,67.063860,270.650000,2.850000,-2.250000,-5.500000,0.000000,0.000000,0.100000,-0.173325,0.192650,3.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,1.5,1.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20.050000,0.800000,86.000000,0.0,0.000000,67.384891,8463.691686,172.500000,67.313070,262.250000,3.100000,-2.550000,-6.500000,0.000000,0.000000,0.150000,0.122070,0.249210,4.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...

In [ ]:
de_train_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month,HelmertEncoder_time_0,HelmertEncoder_time_1,HelmertEncoder_time_2,HelmertEncoder_time_3,HelmertEncoder_time_4,HelmertEncoder_time_5,HelmertEncoder_time_6,HelmertEncoder_time_7,HelmertEncoder_time_8,HelmertEncoder_time_9,HelmertEncoder_time_10,HelmertEncoder_time_11,HelmertEncoder_time_12,HelmertEncoder_time_13,HelmertEncoder_time_14,...,BackwardDifferenceEncoder_day_25,BackwardDifferenceEncoder_day_26,BackwardDifferenceEncoder_day_27,BackwardDifferenceEncoder_day_28,BackwardDifferenceEncoder_day_29,BackwardDifferenceEncoder_weekend_0,BackwardDifferenceEncoder_week_of_month_0,BackwardDifferenceEncoder_week_of_month_1,BackwardDifferenceEncoder_week_of_month_2,BackwardDifferenceEncoder_week_of_month_3,CountEncoder_weekday,CountEncoder_month,CountEncoder_week,CountEncoder_day,CountEncoder_weekend,CountEncoder_week_of_month,week_of_month_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,시간간격_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,한전기_차이_Is_Outliers_IQR,한전DI_차이_Is_Outliers_IQR,습도_Is_Outliers_IQR,기온_Is_Outliers_IQR,week_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,weekday_Is_Outliers_IQR,CDH_Is_Outliers_IQR,한전HI_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,일조_Is_Outliers_IQR,강수량_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,풍속_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR
0,20.250000,2.300000,83.000000,0.0,0.000000,68.903137,8463.315857,168.500000,67.465995,294.400000,2.600000,-0.400000,-8.500000,-0.092702,0.000000,-0.550000,-1.506485,-0.746075,0.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,19.850000,1.850000,85.000000,0.0,0.000000,67.722362,8463.409814,169.500000,66.927275,287.500000,2.250000,-1.200000,-6.000000,-0.046351,0.000000,-0.400000,-1.180775,-0.538720,1.5,0.0,6.0,24.0,8.0,0.0,2.0,0.5,0.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,19.800000,1.100000,85.500000,0.0,0.000000,67.436146,8463.503772,170.500000,66.871210,279.400000,2.500000,-2.100000,-5.500000,0.000000,0.000000,-0.050000,-0.286216,-0.056065,2.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,19.900000,1.000000,86.000000,0.0,0.000000,67.262821,8463.597729,171.500000,67.063860,270.650000,2.850000,-2.250000,-5.500000,0.000000,0.000000,0.100000,-0.173325,0.192650,3.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,1.5,1.5,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20.050000,0.800000,86.000000,0.0,0.000000,67.384891,8463.691686,172.500000,67.313070,262.250000,3.100000,-2.550000,-6.500000,0.000000,0.000000,0.150000,0.122070,0.249210,4.5,0.0,6.0,24.0,8.0,0.0,2.0,0.0,0.0,0.0,2.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,-0.8,-0.6,-0.4,-0.2,1440.0,2760.0,840.0,360.0,6720.0,2520.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...

In [ ]:
target_1

0         8583.084
1         8574.984
2         8564.616
3         8554.896
4         8528.976
            ...   
491635    8714.952
491636    8740.224
491637    8730.504
491638    8725.968
491639    8705.232
Name: 전력사용량, Length: 18719, dtype: float64

In [ ]:
avg_target_1

0       8583.084
1       8574.984
2       8564.616
3       8554.896
4       8528.976
          ...   
9354    8731.152
9355    8727.588
9356    8735.364
9357    8728.236
9358    8715.600
Name: 전력사용량, Length: 9359, dtype: float64

In [ ]:
de_test_1

,기온,풍속,습도,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,한전기_차이,한전HI_차이,한전DI_차이,time,weekday,month,week,day,weekend,week_of_month,HelmertEncoder_time_0,HelmertEncoder_time_1,HelmertEncoder_time_2,HelmertEncoder_time_3,HelmertEncoder_time_4,HelmertEncoder_time_5,HelmertEncoder_time_6,HelmertEncoder_time_7,HelmertEncoder_time_8,HelmertEncoder_time_9,HelmertEncoder_time_10,HelmertEncoder_time_11,HelmertEncoder_time_12,HelmertEncoder_time_13,HelmertEncoder_time_14,...,BackwardDifferenceEncoder_day_25,BackwardDifferenceEncoder_day_26,BackwardDifferenceEncoder_day_27,BackwardDifferenceEncoder_day_28,BackwardDifferenceEncoder_day_29,BackwardDifferenceEncoder_weekend_0,BackwardDifferenceEncoder_week_of_month_0,BackwardDifferenceEncoder_week_of_month_1,BackwardDifferenceEncoder_week_of_month_2,BackwardDifferenceEncoder_week_of_month_3,CountEncoder_weekday,CountEncoder_month,CountEncoder_week,CountEncoder_day,CountEncoder_weekend,CountEncoder_week_of_month,week_of_month_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,시간간격_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,한전기_차이_Is_Outliers_IQR,한전DI_차이_Is_Outliers_IQR,습도_Is_Outliers_IQR,기온_Is_Outliers_IQR,week_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,weekday_Is_Outliers_IQR,CDH_Is_Outliers_IQR,한전HI_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,일조_Is_Outliers_IQR,강수량_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,풍속_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR
0,27.800000,1.500000,74.000000,0.0,0.0,87.420247,8639.156876,2040.0,78.602280,342.700000,1.600000,0.600000,-10.000000,0.0,0.000000,-0.300000,-0.891473,-0.327590,0.0,1.0,8.0,35.0,25.0,0.0,5.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,0.2,0.4,0.6,-0.2,1320,2880,120.0,240,6720,960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27.633333,1.366667,75.333333,0.0,0.0,87.091155,8639.250833,2041.0,78.519273,342.000000,1.933333,0.466667,-10.666667,0.0,0.000000,-0.166667,-0.329092,-0.083007,1.0,1.0,8.0,35.0,25.0,0.0,5.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,0.2,0.4,0.6,-0.2,1320,2880,120.0,240,6720,960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,27.466667,1.233333,76.666667,0.0,0.0,86.736660,8639.344790,2042.0,78.431867,341.500000,2.066667,0.633333,-11.333333,0.0,0.000000,-0.166667,-0.354495,-0.087407,2.0,1.0,8.0,35.0,25.0,0.0,5.0,0.0,2.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,0.2,0.4,0.6,-0.2,1320,2880,120.0,240,6720,960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,27.300000,1.100000,78.000000,0.0,0.0,86.355739,8639.438747,2043.0,78.340060,341.200000,2.300000,0.700000,-11.000000,0.0,0.000000,-0.166667,-0.380921,-0.091807,3.0,1.0,8.0,35.0,25.0,0.0,5.0,0.0,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,0.2,0.4,0.6,-0.2,1320,2880,120.0,240,6720,960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26.900000,1.166667,79.666667,0.0,0.0,85.083042,8639.532705,2044.0,77.912697,340.700000,2.100000,0.566667,-10.333333,0.0,0.000000,-0.400000,-1.272697,-0.427363,4.0,1.0,8.0,35.0,25.0,0.0,5.0,0.0,0.0,0.0,4.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,...,-0.16129,-0.129032,-0.096774,-0.064516,-0.032258,-0.5,0.2,0.4,0.6,-0.2,1320,2880,120.0,240,6720,960.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,28.233333,2.800000,72.666667,0.0,0.3,88.7339

In [ ]:
target_1

0         8583.084
1         8574.984
2         8564.616
3         8554.896
4         8528.976
            ...   
491635    8714.952
491636    8740.224
491637    8730.504
491638    8725.968
491639    8705.232
Name: 전력사용량, Length: 18719, dtype: float64

# H20

https://inspaceai.github.io/2019/05/15/H2O-AutoML-Start/

h2o 써봤는데 성능이 그닥이었습니다. ( 5점대 진입을 못했음 )

In [ ]:
!pip install h2o

In [ ]:
import h2o
h2o.init(nthreads=-1) # 2200  2-> 200

In [ ]:
for i in (range(60)):
  print(i+1)
  globals()['h2train_'+str(i+1)] = h2o.H2OFrame(globals()['de_train_'+str(i+1)])
  globals()['h2test_'+str(i+1)] = h2o.H2OFrame(globals()['de_test_'+str(i+1)])
  globals()['h2target_'+str(i+1)] = h2o.H2OFrame(list(globals()['target_'+str(i+1)]))


1
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
2
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
3
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
4
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
5
Parse progress: |█

In [ ]:
h2target_1

In [ ]:
h2train_1

기온,강수량,일조,Heat_Index,전력사용추세,시간간격,Discomfort_Index,CDH,저기_차이,저풍_차이,저습_차이,저강_차이,저일_차이,time,weekday,month,week,day,weekend,week_of_month,holiday,OneHotEncoder_time_1,OneHotEncoder_time_2,OneHotEncoder_time_3,OneHotEncoder_time_4,OneHotEncoder_time_5,OneHotEncoder_time_6,OneHotEncoder_time_7,OneHotEncoder_time_8,OneHotEncoder_time_9,OneHotEncoder_time_10,OneHotEncoder_time_11,OneHotEncoder_time_12,OneHotEncoder_time_13,OneHotEncoder_time_14,OneHotEncoder_time_15,OneHotEncoder_time_16,OneHotEncoder_time_17,OneHotEncoder_time_18,OneHotEncoder_time_19,OneHotEncoder_time_20,OneHotEncoder_time_21,OneHotEncoder_time_22,OneHotEncoder_time_23,OneHotEncoder_time_24,OneHotEncoder_weekday_1,OneHotEncoder_weekday_2,OneHotEncoder_weekday_3,OneHotEncoder_weekday_4,OneHotEncoder_weekday_5,OneHotEncoder_weekday_6,OneHotEncoder_weekday_7,OneHotEncoder_month_1,OneHotEncoder_month_2,OneHotEncoder_month_3,OneHotEncoder_week_1,OneHotEncoder_week_2,OneHotEncoder_week_3,OneHotEncoder_week_4,OneHotEncoder_week_5,OneHotEncoder_week_6,OneHotEncoder_week_7,OneHotEncoder_week_8,OneHotEncoder_week_9,OneHotEncoder_week_10,OneHotEncoder_week_11,OneHotEncoder_week_12,OneHotEncoder_day_1,OneHotEncoder_day_2,OneHotEncoder_day_3,OneHotEncoder_day_4,OneHotEncoder_day_5,OneHotEncoder_day_6,OneHotEncoder_day_7,OneHotEncoder_day_8,OneHotEncoder_day_9,OneHotEncoder_day_10,OneHotEncoder_day_11,OneHotEncoder_day_12,OneHotEncoder_day_13,OneHotEncoder_day_14,OneHotEncoder_day_15,OneHotEncoder_day_16,OneHotEncoder_day_17,OneHotEncoder_day_18,OneHotEncoder_day_19,OneHotEncoder_day_20,OneHotEncoder_day_21,OneHotEncoder_day_22,OneHotEncoder_day_23,OneHotEncoder_day_24,OneHotEncoder_day_25,OneHotEncoder_day_26,OneHotEncoder_day_27,OneHotEncoder_day_28,OneHotEncoder_day_29,OneHotEncoder_day_30,OneHotEncoder_day_31,OneHotEncoder_weekend_1,OneHotEncoder_weekend_2,OneHotEncoder_week_of_month_1,OneHotEncoder_week_of_month_2,OneHotEncoder_week_of_month_3,OneHotEncoder_week_of_month_4,OneHotEncoder_week_of_month_5,OneHotEncoder_holiday_1,OneHotEncoder_holiday_2,SumEncoder_time_0,SumEncoder_time_1,SumEncoder_time_2,SumEncoder_time_3,SumEncoder_time_4,SumEncoder_time_5,SumEncoder_time_6,SumEncoder_time_7,SumEncoder_time_8,SumEncoder_time_9,SumEncoder_time_10,SumEncoder_time_11,SumEncoder_time_12,SumEncoder_time_13,SumEncoder_time_14,SumEncoder_time_15,SumEncoder_time_16,SumEncoder_time_17,SumEncoder_time_18,SumEncoder_time_19,SumEncoder_time_20,SumEncoder_time_21,SumEncoder_time_22,SumEncoder_weekday_0,SumEncoder_weekday_1,SumEncoder_weekday_2,SumEncoder_weekday_3,SumEncoder_weekday_4,SumEncoder_weekday_5,SumEncoder_month_0,SumEncoder_month_1,SumEncoder_week_0,SumEncoder_week_1,SumEncoder_week_2,SumEncoder_week_3,SumEncoder_week_4,SumEncoder_week_5,SumEncoder_week_6,SumEncoder_week_7,SumEncoder_week_8,SumEncoder_week_9,SumEncoder_week_10,SumEncoder_day_0,SumEncoder_day_1,SumEncoder_day_2,SumEncoder_day_3,SumEncoder_day_4,SumEncoder_day_5,SumEncoder_day_6,SumEncoder_day_7,SumEncoder_day_8,SumEncoder_day_9,SumEncoder_day_10,SumEncoder_day_11,SumEncoder_day_12,SumEncoder_day_13,SumEncoder_day_14,SumEncoder_day_15,SumEncoder_day_16,SumEncoder_day_17,SumEncoder_day_18,SumEncoder_day_19,SumEncoder_day_20,SumEncoder_day_21,SumEncoder_day_22,SumEncoder_day_23,SumEncoder_day_24,SumEncoder_day_25,SumEncoder_day_26,SumEncoder_day_27,SumEncoder_day_28,SumEncoder_day_29,SumEncoder_weekend_0,SumEncoder_week_of_month_0,SumEncoder_week_of_month_1,SumEncoder_week_of_month_2,SumEncoder_week_of_month_3,SumEncoder_holiday_0,시간간격_Is_Outliers_IQR,저강_차이_Is_Outliers_IQR,month_Is_Outliers_IQR,weekday_Is_Outliers_IQR,강수량_Is_Outliers_IQR,week_of_month_Is_Outliers_IQR,일조_Is_Outliers_IQR,week_Is_Outliers_IQR,전력사용추세_Is_Outliers_IQR,기온_Is_Outliers_IQR,CDH_Is_Outliers_IQR,저풍_차이_Is_Outliers_IQR,day_Is_Outliers_IQR,저습_차이_Is_Outliers_IQR,저일_차이_Is_Outliers_IQR,time_Is_Outliers_IQR,Discomfort_Index_Is_Outliers_IQR,Heat_Index_Is_Outliers_IQR,저기_차이_Is_Outliers_IQR
20.6,0,0,70.0497,8463.27,168,67.9221,-14.5,3,-0.5,-11,-0.10486

In [ ]:
len(h2target_1)

9360

In [ ]:
%%time
from h2o.automl import H2OAutoML

# target["C1"] = target["C1"].asnumeric()

predictors = list(h2train_1.columns) 
numeric_response = "C1"

for i in range(60):
  globals()['h2conbined_'+str(i+1)] = globals()['h2train_'+str(i+1)].cbind(globals()['h2target_'+str(i+1)])

# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
for i in range(60):
  
  globals()['h2auto_'+str(i+1)] = H2OAutoML(max_models=40, seed=42, max_runtime_secs=800,nfolds = 15)
  # print(i+1)

for i in range(60):
  globals()['h2auto_'+str(i+1)].train(x=predictors, y=numeric_response, training_frame=globals()['h2conbined_'+str(i+1)])

# View the AutoML Leaderboard
lb = h2auto_1.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

# AutoML Opt

최적화 부분 출력은 길이를 너무 차지해서 줄이겠습니다.

출력예시)
```
17:08:52 | regression optimize: minimize
17:08:52 | ##################################################
55
17:08:52 | > Step 1: calc parameters and pruned score: get test 10 trials
17:10:06 |  One iteration ~ 7.4 sec
17:10:06 |  Possible iters ~ 149.0
17:10:06 | --------------------------------------------------
17:10:06 |   Pruned Threshold Score: 28.4153
17:10:06 | ##################################################
17:10:06 | > Step 2: Full opt with Threshold Score Pruner
17:10:06 | ##################################################
17:10:06 | > Start optimization with the parameters:
17:10:06 | CV_Folds = 35
17:10:06 | Score_CV_Folds = 2
17:10:06 | Feature_Selection = False
17:10:06 | Opt_lvl = 3
17:10:06 | Cold_start = 120
17:10:06 | Early_stoping = 120
17:10:06 | Metric = mean_absolute_error
17:10:06 | Direction = minimize
17:10:06 | ##################################################

Optimize: : 57it [12:48, 13.49s/it, | Model: LightGBM | OptScore: 10.1031 | Best mean_absolute_error: 5.3794 ]

17:22:55 | > Finish Opt!
```

In [ ]:
from automl_alex import LightGBMRegressor
from sklearn.metrics import mean_absolute_error
import sklearn
model = LightGBMRegressor(random_state=42)

In [ ]:
for i in range(60):
  print(i+1)
  # globals()['de_train_'+str(i+1)] = de.fit_transform(globals()['train_'+str(i+1)])
  model.opt( globals()['de_train_'+str(i+1)],  globals()['target_'+str(i+1)],
            verbose=3,  cold_start=120,folds=35,opt_lvl=3,early_stoping=120, auto_parameters=False,#metric=sklearn.metrics.roc_auc_score,
      timeout=1100,metric=sklearn.metrics.mean_absolute_error # optimization time in seconds,
      )

  # cv = CrossValidation(
  #         estimator=model,
  #         folds=15,
  #         score_folds=5,
  #         n_repeats=1,
  #         metric=sklearn.metrics.mean_absolute_error,
  #         print_metric=False, 
  #         metric_round=4, 
  #         random_state=42
  #         )

  # score, score_std = cv.fit_score(globals()['de_train_'+str(i+1)], globals()['target_'+str(i+1)], print_metric=True)
  # cv.fit(globals()['de_train_'+str(i+1)], globals()['target_'+str(i+1)])

  globals()['predicts_LGBM_'+str(i+1)] = model.predict(globals()['de_test_'+str(i+1)])
  # globals()['predicts_CatB_'+str(i+1)] = cv.predict_test(globals()['de_test_'+str(i+1)])

In [ ]:
predicts_LGBM = pd.DataFrame()
# predicts_CatB = pd.DataFrame()
for i in range(60):
  predicts_LGBM = pd.concat([predicts_LGBM,pd.DataFrame(globals()['predicts_LGBM_'+str(i+1)])],axis=0,ignore_index=True)
  # predicts_CatB = pd.concat([predicts_LGBM,pd.DataFrame(globals()['predicts_CatB_'+str(i+1)])],axis=0,ignore_index=True)


# predicts_LGBM = np.array()
# for i in tqdm(range(60)): # 노이즈로 데이터량 증가
#   np.concatenate((predicts_LGBM, globals()['predicts_LGBM_'+str(i+1)]), axis = 0)

In [ ]:
predicts_LGBM

In [ ]:
# submission['answer'] += predicts_LGBM

submission['answer'] = predicts_LGBM
# submission['answer'] = predicts_CatB

In [ ]:
submission

num_cols = ['기온', '풍속', '습도', '강수량', '일조','Heat_Index','전력사용추세','시간간격','Discomfort_Index','CDH',\
            '저기_차이',	'저풍_차이',	'저습_차이',	'저강_차이',	'저일_차이','한전기_차이','한전HI_차이','한전DI_차이'] # ,'CDrain','CDlight'] 성능하락 제외

In [ ]:
submission.to_csv('/content/drive/MyDrive/daycon_data/전력사용량예측/'+str(date.today().strftime("%B_%d_%Y"))+'_LGBM_CDHv2_한전기차이_한전HI_한전DI_차이.csv', index=False)

# TabNet 부분

In [ ]:
import optuna
from optuna import Trial, visualization
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold
import torch

여기서 weight_decay 가 너무 작으면 cuda 에러가 발생합니다. 1e-3 이나 1.5*1e-4 로 수정하면, ( 규제가 커질수록 ) 에러가 발생 안하는것 같은 경향성을 느꼈습니다

In [ ]:
import time
def Objective(trial):

    n_d = trial.suggest_int("n_d", 64, 128, step=8)
    n_a = trial.suggest_int("n_a", 64, 128, step=8)

    n_steps = trial.suggest_int("n_steps", 4, 12, step=2)
    gamma = trial.suggest_float("gamma", 1, 4.0, step=0.3)

    n_shared = trial.suggest_int("n_shared", 4, 12, step=2)
    n_independent = trial.suggest_int("n_independent", 4, 12, step=2)
    seed = 42
    momentum = trial.suggest_float("momentum", 0.00, 0.1, step=0.01)

    lambda_sparse = trial.suggest_float("lambda_sparse", 1e-4, 1e-3, step=1e-4)# log=True) # step=1e-4)


    tabnet_params = dict(n_d=n_d, n_a=n_a, 
                        n_steps=n_steps, gamma=gamma, 
                        n_shared=n_shared, n_independent= n_independent, seed = seed, momentum=momentum,
                        lambda_sparse=lambda_sparse, 
                        
                        optimizer_fn=torch.optim.Adam,
                        optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
                    
                        scheduler_params=dict(mode="min",
                                              patience=trial.suggest_int("patienceScheduler",low=200,high=300), # changing sheduler patience to be lower than early stopping patience 
                                              min_lr=1e-5,
                                              factor=0.5,),
                         
                        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
                        verbose=0,
                     ) #early stopping

    kf = KFold(n_splits=4, random_state=42, shuffle=True)
    CV_score_array =[]


    for i,train_index, test_index in enumerate(kf.split(X)):
        start2 = time.time()
        print(i+1,' fold is start...')
        X_train, X_valid = X[train_index], X[test_index]
        y_train, y_valid = y[train_index], y[test_index]
        regressor = TabNetRegressor(**tabnet_params)
        regressor.fit(X_train=X_train, y_train=y_train,
                  eval_set=[(X_valid, y_valid)],
                  patience=600, # patience=trial.suggest_int("patience",low=400,high=600),
                  max_epochs=1200, # max_epochs=trial.suggest_int('epochs', 400, 600),
                  eval_metric=['mae'],batch_size=2240 ,virtual_batch_size = 280)
        CV_score_array.append(regressor.best_cost)
        print("time :", time.time() - start2)  # 현재시각 - 시작시간 = 실행 시간


    avg = np.mean(CV_score_array)
    return avg

이번에 해보면서 가장 많이 시간이 지체되었던 하이퍼파라미터 학습 부분입니다. 

툭하면 개복치처럼 세션다운되는....

끔찍한 점은 한번 쿠다 에러가 발생하면 이후 학습까지 에러가 발생해서

아래 코드에는 try catch 문을 썼지만 그래도 사실상 학습이 중단됩니다.

In [ ]:
%%time
import joblib
# X = train.drop(['target', 'id'], axis=1).values
# y = train['target'].values
# y = y.reshape(-1, 1)

from optuna.samplers import TPESampler
sampler = TPESampler(seed=42)



for i in (range(30)):
  try:
    start3 = time.time()
    print(i+1,'===========================================')
    globals()['study_'+str(i+1)] = optuna.create_study(direction="minimize", study_name='TabNet optimization',sampler=sampler)

    X = globals()['de_train_'+str(i+1)].values
    y = globals()['target_'+str(i+1)].values.reshape(-1, 1)
    globals()['study_'+str(i+1)].optimize(Objective, n_trials=10) 
    
    # globals()['Paramstudy_'+str(i+1)] = globals()['study_'+str(i+1)].best_params
    # print(i+1)
    # print(globals()['study_'+str(i+1)].best_params)
    print("Total time :", time.time() - start3)  # 현재시각 - 시작시간 = 실행 시간
  except:
    print(i+1,'%%%%%%%%%%%% ERROR %%%%%%%%%%%%%%%%')
    pass

for i in (range(30,60)):
  try:
    start4 = time.time()
    print(i+1,'===========================================')
    globals()['study_'+str(i+1)] = optuna.create_study(direction="minimize", study_name='TabNet optimization',sampler=sampler)

    X = globals()['de_train_'+str(i+1)].values
    y = globals()['target_'+str(i+1)].values.reshape(-1, 1)
    globals()['study_'+str(i+1)].optimize(Objective, n_trials=10) 
    
    # globals()['Paramstudy_'+str(i+1)] = globals()['study_'+str(i+1)].best_params
    # print(i+1)
    # print(globals()['study_'+str(i+1)].best_params)
    print("Total time :", time.time() - start4)  # 현재시각 - 시작시간 = 실행 시간
  except:
    print(i+1,'%%%%%%%%%%%% ERROR %%%%%%%%%%%%%%%%')
    pass

In [ ]:
for i in range(60):
  print('\n=======================================',str(i+1))
  print(globals()['study_'+str(i+1)].best_params)

In [ ]:
for i in range(60):
      globals()['Paramstudy_'+str(i+1)] = globals()['study_'+str(i+1)].best_params
      joblib.dump(globals()['Paramstudy_'+str(i+1)], "/content/drive/MyDrive/daycon_data/전력사용량예측/Paramstudy_"+str(i+1)+".pkl") # Paramstudy 저장

In [ ]:
for i in range(60):
  globals()['finalParams'+str(i+1)] = dict(
      # n_d=n_d, n_a=n_a, 
      # n_steps=n_steps, gamma=gamma, 
      # n_shared=n_shared, n_independent= n_independent, seed = seed, momentum=momentum,
      # lambda_sparse=lambda_sparse, 
      n_d=globals()['Paramstudy_'+str(i+1)]['n_d'], 
      n_a=globals()['Paramstudy_'+str(i+1)]['n_a'], 

      n_steps=globals()['Paramstudy_'+str(i+1)]['n_steps'], 
      gamma=globals()['Paramstudy_'+str(i+1)]['gamma'],

      n_shared=globals()['Paramstudy_'+str(i+1)]['n_shared'], 
      n_independent=globals()['Paramstudy_'+str(i+1)]['n_independent'], 
      seed=42,
      momentum=globals()['Paramstudy_'+str(i+1)]['momentum'], 
      
      lambda_sparse=globals()['Paramstudy_'+str(i+1)]['lambda_sparse'], 
      optimizer_fn=torch.optim.Adam,
      optimizer_params=dict(lr=2e-2, weight_decay=1e-5),
      
      scheduler_params=dict(mode="min",
                            patience=globals()['Paramstudy_'+str(i+1)]['patienceScheduler'],
                            min_lr=1e-5,
                            factor=0.5,),
      scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
      verbose=0,
      
      )
  globals()['epochs_'+str(i+1)] = globals()['Paramstudy_'+str(i+1)]['epochs']

In [ ]:
for i in range(60):
  print(str(i+1),"==========")
  start = time.time()
  X = globals()['de_train_'+str(i+1)].values
  y = globals()['target_'+str(i+1)].values.reshape(-1, 1)
  regressor = TabNetRegressor(**globals()['finalParams'+str(i+1)]) # final param = param study
  regressor.fit(X_train=X, y_train=y,
            patience=600, max_epochs=1200,
            eval_metric=['mae'],batch_size=2240,virtual_batch_size =280)
  
  globals()['TABpred_'+str(i+1)] =regressor.predict(globals()['de_test_'+str(i+1)].values)

  print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간